## Advanced Lane Finding Project

The goals / steps of this project are the following:

1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
1. Apply a distortion correction to raw images.
1. Apply a perspective transform to rectify binary image ("birds-eye view").
1. Use color transforms, gradients, etc., to create a thresholded binary image.
1. Detect lane pixels and fit to find the lane boundary.
1. Determine the curvature of the lane and vehicle position with respect to center.
1. Warp the detected lane boundaries back onto the original image.
1. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

*Author:* Brahm Windeler  
*Date:* March 9, 2017

## GOAL 1: Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

In [ ]:
# 0-0 IMPORT LIBRARIES

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from ipywidgets import *
from IPython.display import display

from matplotlib.patches import Polygon

In [ ]:
# 1-1 Read in the list of calibration image files

cal_images = glob.glob('RaspiWideAngleCalibrationImages/cali*.jpg')
#print (cal_images)

In [ ]:
# 1-2 Accumulate the calibration points and show the results

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Step through the list and search for chessboard corners
plt.figure(figsize=(20,30))
i = 1

dst_test_images = glob.glob('test_images/*.jpg')
test_img = cv2.imread(dst_test_images[0])

for fname in cal_images:
    
    print ("processing image {}".format(i))

    #img = mpimg.imread(fname)
    ##print ("img ",img.shape)
    ##plt.imshow(img)
    ##plt.show()
    #padding = 500
    #black = np.zeros((img.shape[0]+padding*2,img.shape[1]+padding*2,3),np.uint8)
    ##print ("black ", black.shape)
    #black[padding:(img.shape[0]+padding),padding:(img.shape[1]+padding),:] = img
    #img = black
    ##plt.imshow(black)
    ##plt.show()

    width = int(test_img.shape[0])
    height = int(test_img.shape[1])
    
    img = mpimg.imread(fname)
    img_sm = cv2.resize(img, (height, width))
    img = img_sm
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
    altered = False

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        altered = True

    # show all images, even those that didn't have 9x6 corners
    ax = plt.subplot(7,3,i)
    ax.imshow(img)
    if (altered):
        ax.set_title("9x6 FOUND")
    else:
        ax.set_title("9x6 NOT FOUND")
    i += 1
    
plt.show()

## GOAL 2: Apply a distortion correction to raw images.

In [ ]:
# 2-1 Define a wrapper around the calibration and undistort functions for
#     undistorting an image.

def cal_undistort(img):
    '''
    Undistorts an image given previously determined calibration points.
    '''
    global objpoints, imgpoints
    
    img_size = (img.shape[0], img.shape[1])
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    #return cv2.undistort(img, mtx, dist, None, mtx)

    h,  w = img.shape[:2]
    newCameraMtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newCameraMtx,(w,h),5)
    dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)
    
    return dst
    
    print ("dst.shape: ", dst.shape)
    #x,y,w,h = roi
    #x = 0
    #y = 1694
    #w = 250
    #h = 250
    #print (roi)
    
    x = 150
    w = 850
    y = 300
    h = 450

    dst = dst[y:y+h, x:x+w]
    
    return dst

In [ ]:
def cal_undistort_simple(img):
    '''
    Undistorts an image given previously determined calibration points.
    '''
    global objpoints, imgpoints
    
    img_size = (img.shape[0], img.shape[1])
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    return cv2.undistort(img, mtx, dist, None, mtx)

In [ ]:
# 2-3 Read in test images provided with the project

test_images = glob.glob('test_images/*.jpg')

In [ ]:
i=0
for fname in test_images:
    img = mpimg.imread(fname)
    undistorted = cal_undistort(img)
    mpimg.imsave("undistorted_image{}.jpg".format(i), undistorted)
    i += 1

In [ ]:
# 2-2 Demonstrate how the function undistorts the calibration images

def demonstrate_image_undistort(imgs):
    
    for fname in imgs:
        
        # Read in the image and undistort it
        
        img = mpimg.imread(fname)
        #print ("img ",img.shape)
        #plt.imshow(img)
        #plt.show()
        
        #padding = 500
        #black = np.zeros((img.shape[0]+padding*2,img.shape[1]+padding*2,3),np.uint8)
        #print ("black ", black.shape)
        #black[padding:(img.shape[0]+padding),padding:(img.shape[1]+padding),:] = img
        #img = black
        
        #img = mpimg.imread(fname)
        undistorted = cal_undistort(img)

        # Display the original and undistorted images
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(img)
        ax1.set_title('Original Image', fontsize=50)
        ax2.imshow(undistorted)
        ax2.set_title('Undistorted Image', fontsize=50)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        plt.show()

# UNCOMMENT TO RUN     
#demonstrate_image_undistort([cal_images[0]])

In [ ]:
# 2-4 Demonstrate undistortion of some test images

# UNCOMMENT TO RUN     
demonstrate_image_undistort(test_images[:])

## Obtain images for testing

In [ ]:
# 2-5 Define some generators for extracting frames from video clips.
#     We'll need some consecutive images for testing previous fit lines.

from moviepy.editor import VideoFileClip
import distutils.dir_util

def frange(start, stop, step):
    ''' Generator: Converts start, stop, and step parameters to a time value. '''
    _idx = start
    while _idx < stop:
        yield _idx
        _idx += step

def extract_frames(name, start_time=0.0, interval=0.1, max_images=25):
    ''' Generator: Extracts frames from a video clip and returns them one at a time. '''
    if (name == ""):
        return
    clip = VideoFileClip(name + '.mp4')
    for _clip_idx in frange(start_time, min([clip.end, start_time + max_images*interval]), interval):
        yield clip.to_ImageClip(_clip_idx).img

def save_frames_to_dir(name, t_start=0, t_end=None, fps=24):
    ''' Extracts frames from a video clip and saves them to the filesystem. '''
    if (name == ""):
        return
    clip = VideoFileClip(name + '.mp4').subclip(t_start, t_end)
    distutils.dir_util.mkpath('{}/'.format(name))
    return clip.write_images_sequence("{}/frame%03d.jpg".format(name), fps=fps)

In [ ]:
# 2-6 Test the frame extracting generators by extracting a few frames.

def demonstrate_extracting_video_frames():
    for img in extract_frames('project_video', 0.0, 0.1, 3):
        plt.figure()
        plt.imshow(img)
        plt.show()
        
# UNCOMMENT TO RUN
#demonstrate_extracting_video_frames()

In [ ]:
# 2-7 Test the frame saving function

# UNCOMMENT TO RUN
#save_frames_to_dir('project_video', t_start=22.2, t_end=23.2, fps=24) # Problem spot 1
#save_frames_to_dir('project_video', t_start=41.2, t_end=41.7, fps=24) # Problem spot 2

In [ ]:
# 2-8 Read in the extracted project video frames 

#pv_images = glob.glob('project_video/*.jpg')

## GOAL 3: Apply a perspective transform to rectify binary image ("birds-eye view").

In [ ]:
# 3-1 Define a class to handle performing a birds-eye transform of the lane ahead.
#     Also contains the src and dst reference points as well as methods for drawing
#     reference boundaries on the transformed images.

class BirdsEyeTransform():
    ''' Class to hold the birds-eye transform information. Assumes 720 H x 1280 W 
        image from the center forward facing camera.
    '''
    def __init__(self):
        # Reference source points for the perspective transform
        #self.src = np.float32([(200,719), (580,460), (706,460), (1109,719)]) # Not symetric
        ###self.src = np.float32([(200,719), (580,460), (699,460), (1079,719)])
        #self.src = np.float32([(200,719), (595,450), (684,450), (1079,719)]) # Too blurry at top when warped?

        #                          TL       TR          BR         BL
        self.src = np.float32([(406,431), (754,431), (974,674), (186,674)])
        
        # Reference destination points for the perspective transform
        ###self.dst = np.float32([(320,719), (320,0),   (960,0),    (960,719)])
        self.dst = np.float32([(320,0),   (960,0),    (960,994), (320,994)])
    
        
        # The transformation matrix for the perspective warp
        self.M = cv2.getPerspectiveTransform(self.src, self.dst)
        
        # Inverse transformation matrix for the perspective warp
        self.Minv = cv2.getPerspectiveTransform(self.dst, self.src)
        
        # Points defining the ROI from the src img
        src_shape = (994, 1296)
        img = np.ones(src_shape, dtype=np.uint8)*255
        warped = cv2.warpPerspective(img, self.Minv, (img.shape[1], img.shape[0]))
        src_img = np.rint(warped).astype(np.uint8)
    
        src_img_nonzeros = src_img.nonzero()
        x_min = min(src_img_nonzeros[1])
        x_max = max(src_img_nonzeros[1])
        y_min = min(src_img_nonzeros[0])
        y_max = max(src_img_nonzeros[0])
            
        poly = []
        poly.append((x_min, y_max))
        poly.append((x_min, min(src_img_nonzeros[0][src_img_nonzeros[1] == x_min])))
        poly.append((min(src_img_nonzeros[1][src_img_nonzeros[0] == y_min]), y_min))
        poly.append((max(src_img_nonzeros[1][src_img_nonzeros[0] == y_min]), y_min))
        poly.append((x_max, min(src_img_nonzeros[0][src_img_nonzeros[1] == x_max])))
        poly.append((x_max, y_max))

        self.roi = np.float32(poly)
        
    def draw_src_on_img(self, img, color=(255,0,0), thickness=2):
        ''' Takes an RGB image and draws the src points (trapezoid) directly on
            the image.
        '''
        cv2.polylines(img, [self.src.astype(int)], True, color=color, thickness=thickness)
        
    def draw_dst_on_img(self, img, color=(255,0,0), thickness=2):
        ''' Takes an RGB image and draws the dst points (square) directly on
            the image.
        '''
        # Perspective transforms require float points, but polylines requires ints
        cv2.polylines(img, [self.dst.astype(int)], True, color=color, thickness=thickness)

    def draw_src_on_img_gray(self, img, intensity=255, thickness=2):
        ''' Takes a grayscale image and draws the src points (trapezoid) directly on
            the image.
        '''
        cv2.polylines(img, [self.src.astype(int)], True, color=intensity, thickness=thickness)
        
    def draw_dst_on_img_gray(self, img, intensity=255, thickness=2):
        ''' Takes a grayscale image and draws the dst points (square) directly on
            the image.
        '''
        # Perspective transforms require float points, but polylines requires ints
        cv2.polylines(img, [self.dst.astype(int)], True, color=intensity, thickness=thickness)
        
    def warp(self, img):
        ''' Performs the perspective warp from src to dst. Returns the result. '''

        warped = cv2.warpPerspective(img, self.M, (img.shape[1], img.shape[0]))
        
        # Round floats produced by warp to ints, then convert to unsigned 8-bit
        # https://carnd-forums.udacity.com/questions/38545026/black-images
        return np.rint(warped).astype(np.uint8)
    
    def unwarp(self, img):
        ''' Performs the perspective warp from dst back to src. Returns the result. '''
        warped = cv2.warpPerspective(img, self.Minv, (img.shape[1], img.shape[0]))
        return np.rint(warped).astype(np.uint8)
    
    def apply_cropping_mask(self, img):
        
        # get dimensions of mask
        bottom_right_pt = self.src[2]
        bottom_left_pt = self.src[3]
        offset = 86
        bottom_right_x = int(bottom_right_pt[0] + offset)
        bottom_left_x = int(bottom_left_pt[0] - offset)

        # apply mask
        #img[:,0:bottom_left_x,:] = img[:,bottom_left_x:bottom_left_x+1,:]
        #img[:,bottom_right_x:,:] = img[:,bottom_right_x:bottom_right_x+1,:]
        img[:,0:bottom_left_x,:] = 0
        img[:,bottom_right_x:,:] = 0

        return img

In [ ]:
# 3-2 Create an instance of the BirdsEyeTransform class.

birdseye = BirdsEyeTransform()

In [ ]:
# 3-3 Visualize the birdseye transform on test images

def demonstrate_birdseye_transform(imgs):

    for fname in imgs:

        img = mpimg.imread(fname)
        undistorted = cal_undistort(img)
        
        # Pre-process the image by masking out stuff
        undistorted_masked = birdseye.apply_cropping_mask(undistorted)
        
        undistorted_warped = birdseye.warp(undistorted_masked)

        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 22))
        f.tight_layout()

        ax1.imshow(undistorted)
        ax1.set_title('Undistorted', fontsize=30)
        ax1.add_patch(Polygon(birdseye.src, True, edgecolor='#ff0000', fill=False))

        ax2.imshow(undistorted_warped)
        ax2.set_title('Undistorted -> Warped', fontsize=30)
        ax2.add_patch(Polygon(birdseye.dst, True, edgecolor='#ff0000', fill=False))

        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        plt.show()

# UNCOMMENT TO RUN
demonstrate_birdseye_transform(test_images[:])

In [ ]:
# 3-4 Verify how much the center shifts for the birds-eye transform

# Assuming the left and right sides of the src and dst are symetrical and
# equidistant from the sides of the image, the center shouldn't shift
# (This was a problem with my originally chosen points.)

def demonstrate_center_shift():
    
    src_img = np.zeros((720, 1280), dtype=np.uint8)
    
    center_marker = np.int32([(src_img.shape[1]//2 - 1,0), (src_img.shape[1]//2 - 1,src_img.shape[0])])
    print("center_marker", center_marker)
    cv2.polylines(src_img, [center_marker], False, 255, 1)
    
    dst_img = birdseye.warp(src_img)
    
    dst_bottom_nonzeros = dst_img[dst_img.shape[0]-1].nonzero()
    print("dst_bottom_nonzeros", dst_bottom_nonzeros)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))
    f.tight_layout()

    birdseye.draw_src_on_img_gray(src_img, intensity=127, thickness=2)
    ax1.imshow(src_img, cmap='gray')
    
    birdseye.draw_dst_on_img_gray(dst_img, intensity=127, thickness=2)
    ax2.imshow(dst_img, cmap='gray')
    
    plt.show()

# UNCOMMENT TO RUN
demonstrate_center_shift()

In [ ]:
# 3-5 Visualize how a birdseye transform looks when "unwarped" back to the 
#     original perspective.

def demonstrate_birdseye_reverse_usage():
    
    # Create a blank white image that represents the result of a birdeye warp
    dst_img = np.ones((720, 1280), dtype=np.uint8)*255
    
    # "Unwarp" the image to the aleged source
    src_img = birdseye.unwarp(dst_img)

    # Determine the bounding box of the non-zero values from the unwarped image
    src_img_nonzeros = src_img.nonzero()
    x_min = min(src_img_nonzeros[1])
    x_max = max(src_img_nonzeros[1])
    y_min = min(src_img_nonzeros[0])
    y_max = max(src_img_nonzeros[0])
    
    #print("src_img_nonzeros", src_img_nonzeros)
    
    print("{} {} {} {}".format(x_min, x_max, y_min, y_max))

    # Get the polygon points that define the "unwarped" birdseye view
    poly = []
    poly.append((x_min, y_max))
    poly.append((x_min, min(src_img_nonzeros[0][src_img_nonzeros[1] == x_min])))
    poly.append((min(src_img_nonzeros[1][src_img_nonzeros[0] == y_min]), y_min))
    poly.append((max(src_img_nonzeros[1][src_img_nonzeros[0] == y_min]), y_min))
    poly.append((x_max, min(src_img_nonzeros[0][src_img_nonzeros[1] == x_max])))
    poly.append((x_max, y_max))
    print("poly", poly)
    
    # Display the results
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))
    f.tight_layout()

    # Draw the simulated birdseye
    birdseye.draw_dst_on_img_gray(dst_img, intensity=127, thickness=2)
    ax1.imshow(dst_img, cmap='gray')
    
    # Draw the simulated original source image with the birdseye src coords and
    # the polygon defining how the original was unwarped
    birdseye.draw_src_on_img_gray(src_img, intensity=127, thickness=2)
    ax2.imshow(src_img, cmap='gray')
    ax2.add_patch(Polygon(np.float32(poly), True, edgecolor='#ff0000', fill=False))
    
    plt.show()
    
# UNCOMMENT TO RUN
#demonstrate_birdseye_reverse_usage()

## GOAL 4: Use color transforms, gradients, etc., to create a thresholded binary image.

In [ ]:
# 4-1 Define a number of threshold filters that can be used to try to
#     extract the lane lines

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    '''
    Applies Sobel filter in the x or y direction, takes the absolute
    value and applies the specified threshold.
    '''
    
    # 1) Convert to grayscale if necessary
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img

    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    #    is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    # 6) Return this mask as the binary_output image
    return binary_output


def grad_magnitude_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    '''
    Applies a Sobel in the x and y directions, computes the magnitude
    of the gradient, then applies the specified threshold
    '''
    
    # 1) Convert to grayscale if necessary
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img
    
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Calculate the magnitude
    abs_sobel = np.power(np.add(np.power(sobelx,2),np.power(sobely,2)), 0.5)
    
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # 5) Create a binary mask where magnitude thresholds are met
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    # 6) Return this mask as the binary_output image
    return binary_output


def grad_direction_thresh(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    '''
    Applies a Sobel in the x and y directions, then computes the
    direction of the gradient and applies the specified threshold.
    '''
    
    # 1) Convert to grayscale if necessary
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img
    
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    dir_sobel = np.arctan2(abs_sobely, abs_sobelx)
    
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(dir_sobel)
    binary_output[(dir_sobel >= thresh[0]) & (dir_sobel <= thresh[1])] = 1
    
    # 6) Return this mask as the binary_output image
    return binary_output

def color_thresh(img, cvt=cv2.COLOR_RGB2HLS, channel=2, thresh=(0,255)):
    ''' Selects a single color channel and applies a min/max threshold
        to the values, returning a binary mask for the pixels that meet
        the specified thresholds.
        By default, converts from RGB to HLS space and uses the Saturation
        channel.
    '''
    
    # 1) Convert to the specified colorspace
    if cvt != False:
        alt = cv2.cvtColor(img, cvt)
    else:
        alt = img
    
    # 2) Select the specified channel from the colorspace
    single_channel = alt[:,:,channel]
    
    # 3) Create a binary mask where the channel thresholds are met
    binary_output = np.zeros_like(single_channel)
    binary_output[(single_channel >= thresh[0]) & (single_channel <= thresh[1])] = 1
    
    # 4) Return this mask as the binary_output image
    return binary_output

In [ ]:
# 4-2 Various functions that combine the gradient and color threshold outputs.
#     The first two were strategies that were tested before ultimately
#     finding the last to be the most robust.

# Does not use magnitude or direction, as they seemed to be very noisy
def combined_thresh_strategy1(img, grad_ksize=27, mag_ksize=27, dir_ksize=15,
           gradx_thresh=(20, 80), 
           grady_thresh=(30, 80),
           mag_thresh=(15, 175),
           dir_thresh=(0.6, 1.2),
           sat_thresh=(140, 255),
           lum_thresh=(198, 255)):

    '''
    Returns a binary image where the 'on' pixels pass a combination 
    of the above filters using the following logic:
    
    ((GradX & GradY) | Sat | Lum)
    Old: ((GradX & GradY) | (Mag & Dir) | Sat ) & Lum

    '''
    
    gry = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gry, orient='x', sobel_kernel=grad_ksize, thresh=gradx_thresh)
    grady = abs_sobel_thresh(gry, orient='y', sobel_kernel=grad_ksize, thresh=grady_thresh)
    #mag_binary = grad_magnitude_thresh(gry, sobel_kernel=mag_ksize, thresh=mag_thresh)
    #dir_binary = grad_direction_thresh(gry, sobel_kernel=dir_ksize, thresh=dir_thresh)
    sat_binary = color_thresh(hls, cvt=False, thresh=sat_thresh)
    lum_binary = color_thresh(hls, cvt=False, channel=1, thresh=lum_thresh)

    combined = np.zeros_like(sat_binary)
    #combined[(((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | (sat_binary == 1)) & (lum_binary == 1)] = 1
    combined[(((gradx == 1) & (grady == 1)) | (sat_binary == 1)) | (lum_binary == 1)] = 1
    
    return combined

# Adapted from
# https://github.com/swirlingsand/self-driving-car-nanodegree-nd013/blob/master/p4-CarND-Advanced-Lane-Lines/methods/processImage.py
def combined_thresh_strategy2(img, grad_ksize=27, mag_ksize=27, dir_ksize=15,
           gradx_thresh=(10, 120), 
           grady_thresh=(10, 120),
           mag_thresh=(10, 120),
           dir_thresh=(0.7, 2.0),
           sat_thresh=(120, 200)):
    
    gry = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gry, orient='x', sobel_kernel=grad_ksize, thresh=gradx_thresh)
    grady = abs_sobel_thresh(gry, orient='y', sobel_kernel=grad_ksize, thresh=grady_thresh)
    mag_binary = grad_magnitude_thresh(gry, sobel_kernel=mag_ksize, thresh=mag_thresh)
    dir_binary = grad_direction_thresh(gry, sobel_kernel=dir_ksize, thresh=dir_thresh)
    sat_binary = color_thresh(hls, cvt=False, thresh=sat_thresh)

    combined = np.zeros_like(sat_binary)
    combined[( ((gradx == 1)      & (grady == 1))      | 
               ((mag_binary == 1) & (dir_binary == 0)) | 
               (sat_binary == 1)
             )] = 1
    
    return combined

# Ultimately chosen implementation. Uses slightly different gradient parameters than strategy2
# and instead of using the S channel from the HLS color encoding, the Luminosity and 
# Saturation HLS channels are added and averaged. This counteracts dark shadows on the 
# road that would otherwise have high values in the S channel.
# Additionally, the inverse Direction gradient is used as this "lights" up the lane lines
# appropritely for post-birdseye-warped images.
def combined_thresh(img, grad_ksize=27, mag_ksize=27, dir_ksize=15,
           gradx_thresh=(30, 120), 
           grady_thresh=(30, 120),
           mag_thresh=(30, 120),
           dir_thresh=(0.7, 1.57),
           lumsat_thresh=(145, 255)):

    '''
    Returns a binary image where the 'on' pixels pass a combination 
    of the above filters using the following logic:
    
    (GradX & GradY) | (Mag & Dir) | ((Lum+Sat)/2)

    '''

    # Convert the RGB to useful formats
    gry = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gry, orient='x', sobel_kernel=grad_ksize, thresh=gradx_thresh)
    grady = abs_sobel_thresh(gry, orient='y', sobel_kernel=grad_ksize, thresh=grady_thresh)
    mag_binary = grad_magnitude_thresh(gry, sobel_kernel=mag_ksize, thresh=mag_thresh)
    dir_binary = grad_direction_thresh(gry, sobel_kernel=dir_ksize, thresh=dir_thresh)

    # Take the average of adding the L and S channels from the HLS encoding and then apply
    # the appropriate thresholds
    lumsat = (np.float32(hls[:,:,1]) + np.float32(hls[:,:,2]))//2
    lumsat_binary = np.zeros_like(gradx)
    lumsat_binary[(lumsat >= lumsat_thresh[0]) & (lumsat <= lumsat_thresh[1])] = 1

    # Combine the results
    combined = np.zeros_like(dir_binary)
    combined[(((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 0)) | (lumsat_binary == 1))] = 1

    # Convert to unsigned int and return
    return np.uint8(combined)

def davg_thresh(img, lumsat_thresh=(100, 255)):
    # Convert the RGB to useful formats
    gry = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # Take the average of adding the L and S channels from the HLS encoding and then apply
    # the appropriate thresholds
    lumsat = (np.float32(hls[:,:,1]) + np.float32(hls[:,:,2]))//2
    lumsat_binary = np.zeros_like(gry)
    lumsat_binary[(lumsat >= lumsat_thresh[0]) & (lumsat <= lumsat_thresh[1])] = 1

    return np.uint8(lumsat_binary)

In [ ]:
# 4-3 Define some additional diagnostic functions
#     Adapted from https://carnd-forums.udacity.com/questions/32706990/answers/38548228

def normalized(img):
    return np.uint8(255*img/np.max(np.absolute(img)))

def to_RGB(img):
    if img.ndim == 2:
        img_normalized = normalized(img)
        return np.dstack((img_normalized, img_normalized, img_normalized))
    elif img.ndim == 3:
        return img
    else:
        return None
    
def update_diagScreen_cell(diagScreen, img, w, h, r, c, text=None, color=(255,0,0), thickness=2):
    rgb = to_RGB(img)
    if text is not None:
        fontFace = cv2.FONT_HERSHEY_COMPLEX
        fontScale = 2
        offset = 20
        textSize, _ = cv2.getTextSize(text, fontFace, fontScale, thickness)
        cv2.putText(rgb, text, (offset, textSize[1] + offset), fontFace, fontScale, color, thickness)
    diagScreen[r*h:(r+1)*h, c*w:(c+1)*w] = cv2.resize(rgb, (w, h), interpolation=cv2.INTER_AREA)
    return diagScreen

def compose_filter_diagScreen(diag1=None, diag2=None, diag3=None, 
                              diag4=None, diag5=None, diag6=None, 
                              diag7=None, diag8=None, diag9=None,
                              diag10=None, diag11=None, diag12=None,
                              diag13=None, diag14=None, diag15=None,
                              title1=None, title2=None, title3=None, 
                              title4=None, title5=None, title6=None,
                              title7=None, title8=None, title9=None,
                              title10=None, title11=None, title12=None,
                              title13=None, title14=None, title15=None):

    width = 320
    height = 240
    
    rows = 5
    cols = 3
    
    # Initialize the output image.
    diagScreen = np.zeros((height * rows, width * cols, 3), dtype=np.uint8)
    
    # top row
    if diag1 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag1, width, height, r=0, c=0, text=title1)
    if diag2 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag2, width, height, r=0, c=1, text=title2)
    if diag3 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag3, width, height, r=0, c=2, text=title3)

    if diag4 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag4, width, height, r=1, c=0, text=title4)
    if diag5 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag5, width, height, r=1, c=1, text=title5)
    if diag6 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag6, width, height, r=1, c=2, text=title6)

    if diag7 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag7, width, height, r=2, c=0, text=title7)
    if diag8 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag8, width, height, r=2, c=1, text=title8)
    if diag9 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag9, width, height, r=2, c=2, text=title9)

    if diag10 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag10, width, height, r=3, c=0, text=title10)
    if diag11 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag11, width, height, r=3, c=1, text=title11)
    if diag12 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag12, width, height, r=3, c=2, text=title12)

    # bottom row
    if diag13 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag13, width, height, r=4, c=0, text=title13)
    if diag14 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag14, width, height, r=4, c=1, text=title14)
    if diag15 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag15, width, height, r=4, c=2, text=title15)

    return diagScreen

def compose_2x3_screen(diag1=None, diag2=None, diag3=None, 
                       diag4=None, diag5=None, diag6=None, 
                       title1=None, title2=None, title3=None, 
                       title4=None, title5=None, title6=None, 
                       color=(255,0,0), thickness=2):

    width = 320
    height = 240
    
    rows = 2
    cols = 3
    
    # Initialize the output image.
    diagScreen = np.zeros((height * rows, width * cols, 3), dtype=np.uint8)
    
    if diag1 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag1, width, height, r=0, c=0, text=title1, color=color, thickness=thickness)
    if diag2 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag2, width, height, r=0, c=1, text=title2, color=color, thickness=thickness)
    if diag3 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag3, width, height, r=0, c=2, text=title3, color=color, thickness=thickness)

    if diag4 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag4, width, height, r=1, c=0, text=title4, color=color, thickness=thickness)
    if diag5 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag5, width, height, r=1, c=1, text=title5, color=color, thickness=thickness)
    if diag6 is not None:
        diagScreen = update_diagScreen_cell(diagScreen, diag6, width, height, r=1, c=2, text=title6, color=color, thickness=thickness)

    return diagScreen

In [ ]:
# 4-4 Demonstrate the gradient threshold filters

def demonstrate_gradient_threshold_comparison(fname, 
                                    grad_ksize=27, mag_ksize=27, dir_ksize=15,
                                    gradx_min=30, gradx_max=120, 
                                    grady_min=30, grady_max=120,
                                    mag_min=25, mag_max=120,
                                    dir_min=0.7, dir_max=np.pi/2):
    img = mpimg.imread(fname)
    img = cal_undistort(img)
    
    masked = birdseye.apply_cropping_mask(img)   
    img = birdseye.warp(masked)
    
    gry = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gry_rgb = cv2.cvtColor(gry, cv2.COLOR_GRAY2RGB)

    gradx = abs_sobel_thresh(gry, orient='x', sobel_kernel=grad_ksize, thresh=(gradx_min, gradx_max))
    grady = abs_sobel_thresh(gry, orient='y', sobel_kernel=grad_ksize, thresh=(grady_min, grady_max))
    mag_binary = grad_magnitude_thresh(gry, sobel_kernel=mag_ksize, thresh=(mag_min, mag_max))
    dir_binary = grad_direction_thresh(gry, sobel_kernel=dir_ksize, thresh=(dir_min, dir_max))
    
    screen = compose_2x3_screen(diag1=img, diag2=gradx, diag3=grady, 
                       diag4=gry_rgb, diag5=mag_binary, diag6=dir_binary, 
                       title1="Original", title2="Sobel X", title3="Sobel Y", 
                       title4="Grayscale", title5="Magnitude", title6="Direction")
    plt.figure(figsize=(12,12))
    plt.imshow(screen)
    plt.axis('off')
    plt.show()
    
# UNCOMMENT TO RUN
demonstrate_gradient_threshold_comparison(test_images[0])

In [ ]:
# 4-5 Demonstrate the HSV and HLS color channels

def demonstrate_hsv_hls_color_channels(fname):
    
    img = mpimg.imread(fname)
    img = cal_undistort(img)

    masked = birdseye.apply_cropping_mask(img)   
    img = birdseye.warp(masked)

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    hsv0 = cv2.cvtColor(hsv[:,:,0], cv2.COLOR_GRAY2RGB)
    hsv1 = cv2.cvtColor(hsv[:,:,1], cv2.COLOR_GRAY2RGB)
    hsv2 = cv2.cvtColor(hsv[:,:,2], cv2.COLOR_GRAY2RGB)
    
    hls0 = cv2.cvtColor(hls[:,:,0], cv2.COLOR_GRAY2RGB)
    hls1 = cv2.cvtColor(hls[:,:,1], cv2.COLOR_GRAY2RGB)
    hls2 = cv2.cvtColor(hls[:,:,2], cv2.COLOR_GRAY2RGB)
    
    screen = compose_2x3_screen(diag1=hsv0, diag2=hsv1, diag3=hsv2, 
                              diag4=hls0, diag5=hls1, diag6=hls2, 
                              title1="HSV H", title2="HSV S", title3="HSV V", 
                              title4="HLS H", title5="HLS L", title6="HLS S")

    plt.figure(figsize=(12,12))
    plt.imshow(screen)
    plt.axis('off')
    plt.show()
    
# UNCOMMENT TO RUN
demonstrate_hsv_hls_color_channels(test_images[0])

In [ ]:
# 4-6 Demonstrate the HSV and HLS color channels

def demonstrate_hsv_hls_color_channel_adding(fname):
    
    img = mpimg.imread(fname)
    img = cal_undistort(img)

    masked = birdseye.apply_cropping_mask(img)   
    img = birdseye.warp(masked)
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    hsv1 = cv2.cvtColor(hsv[:,:,1], cv2.COLOR_GRAY2RGB)
    hsv2 = cv2.cvtColor(hsv[:,:,2], cv2.COLOR_GRAY2RGB)
    
    hls1 = cv2.cvtColor(hls[:,:,1], cv2.COLOR_GRAY2RGB)
    hls2 = cv2.cvtColor(hls[:,:,2], cv2.COLOR_GRAY2RGB)
    
    satval = cv2.cvtColor(np.uint8((np.float32(hsv[:,:,1]) + np.float32(hsv[:,:,2]))//2), cv2.COLOR_GRAY2RGB)
    lumsat = cv2.cvtColor(np.uint8((np.float32(hls[:,:,1]) + np.float32(hls[:,:,2]))//2), cv2.COLOR_GRAY2RGB)
    
    screen = compose_2x3_screen(diag1=hsv1, diag2=hsv2, diag3=satval, 
                              diag4=hls1, diag5=hls2, diag6=lumsat, 
                              title1="HSV S", title2="HSV V", title3="HSV (S+V)/2", 
                              title4="HLS L", title5="HLS S", title6="HLS (L+S)/2")

    plt.figure(figsize=(12,12))
    plt.imshow(screen)
    plt.axis('off')
    plt.show()
    
# UNCOMMENT TO RUN
demonstrate_hsv_hls_color_channel_adding(test_images[0])

In [ ]:
# 4-7 Visualize the combination of filters on a sample image
#     to allow us to determing good threshold values.

# OLD
def update_demo_basic(imgs=[], grad_ksize=3, mag_ksize=3, dir_ksize=3, 
           gradx_min=0, gradx_max=255, 
           grady_min=0, grady_max=255,
           mag_min=0, mag_max=255,
           dir_min=0, dir_max=np.pi/2,
           sat_min=0, sat_max=255,
           lum_min=0, lum_max=255,
           lumsat_min=0, lumsat_max=255):

    if (len(imgs) == 0):
        print("Must specifiy array of image files.")
        return
    
    img = mpimg.imread(imgs)
    img = cal_undistort(img)
    
    combined = combined_thresh_strat2(img, grad_ksize, mag_ksize, dir_ksize,
           gradx_thresh=(gradx_min, gradx_max), 
           grady_thresh=(grady_min, grady_max),
           mag_thresh=(mag_min, mag_max),
           dir_thresh=(dir_min, dir_max),
           sat_thresh=(sat_min, sat_max),
           lum_thresh=(lum_min, lum_max))
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original ({})'.format(test_images[idx]), fontsize=50)
    ax2.imshow(combined, cmap='gray')
    ax2.set_title('Combined Filter', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

# A comprehensive selection of parameters
def update_demo_thorough(imgs=[], do_warp=False,
           grad_ksize=3, mag_ksize=3, dir_ksize=3,
           gradx_min=0, gradx_max=255, 
           grady_min=0, grady_max=255,
           mag_min=0, mag_max=255,
           dir_min=0, dir_max=np.pi/2,
           sat_min=0, sat_max=255,
           lum_min=0, lum_max=255,
           lumsat_min=0, lumsat_max=255):

    if (len(imgs) == 0):
        print("Must specifiy array of image files.")
        return
    
    img = mpimg.imread(imgs)
    img = cal_undistort(img)
    
    if (do_warp):
        img = birdseye.apply_cropping_mask(img)
        img = birdseye.warp(img)
    
    gry = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gry, orient='x', sobel_kernel=grad_ksize, thresh=(gradx_min, gradx_max))
    grady = abs_sobel_thresh(gry, orient='y', sobel_kernel=grad_ksize, thresh=(grady_min, grady_max))
    mag_binary = grad_magnitude_thresh(gry, sobel_kernel=mag_ksize, thresh=(mag_min, mag_max))
    dir_binary = grad_direction_thresh(gry, sobel_kernel=dir_ksize, thresh=(dir_min, dir_max))
    inv_dir_binary = np.zeros_like(dir_binary)
    inv_dir_binary[dir_binary == 0] = 1
    sat_binary = color_thresh(hls, cvt=False, thresh=(sat_min, sat_max))
    lum_binary = color_thresh(hls, cvt=False, channel=1, thresh=(lum_min, lum_max))

    # Create combined images for some of these
    blank = np.zeros_like(gry).astype(np.uint8)
    grad = np.dstack((blank, gradx & grady, blank))*255
    
    if (do_warp):
        magdir = np.dstack((blank, mag_binary.astype(np.uint8) & inv_dir_binary.astype(np.uint8), blank))*255
    else:
        magdir = np.dstack((blank, mag_binary.astype(np.uint8) & dir_binary.astype(np.uint8), blank))*255
        
    sat = cv2.cvtColor(hls[:,:,2], cv2.COLOR_GRAY2RGB)
    lum = cv2.cvtColor(hls[:,:,1], cv2.COLOR_GRAY2RGB)
    
    lumsat = (np.float32(hls[:,:,1]) + np.float32(hls[:,:,2]))//2
    lumsat_binary = np.zeros_like(lumsat)
    lumsat_binary[(lumsat >= lumsat_min) & (lumsat <= lumsat_max)] = 1

    combined = np.zeros_like(dir_binary)
    if (do_warp):
        combined[(((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (inv_dir_binary == 1)) | (lumsat_binary == 1))] = 1
        dir_img = inv_dir_binary
    else:
        combined[(((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | (lumsat_binary == 1))] = 1
        dir_img = dir_binary
        
    screen = compose_filter_diagScreen(diag1=gradx, title1="Sobel X",
                                       diag2=grad, title2="Sobel X & Y",
                                       diag3=grady, title3="Sobel Y",
                                       diag4=sat, title4="Saturation Orig",
                                       diag5=lumsat, title5="Lum+Sat Orig",
                                       diag6=lum, title6="Luminosity Orig",
                                       diag7=sat_binary, title7="Saturation Thresh",
                                       diag8=lumsat_binary, title8="Lum+Sat Thresh",
                                       diag9=lum_binary, title9="Luminosity Thresh",
                                       diag10=mag_binary, title10="Magnitude",
                                       diag11=magdir, title11="Mag & Dir",
                                       diag12=dir_img, title12="Direction",
                                       diag13=img, title13="Original",
                                       diag14=combined, title14="Used Combined",)

    plt.figure(figsize=(20,20))
    plt.imshow(screen)
    plt.axis('off')
    plt.show()

def demonstrate_filter_parameters(images):
    #imgidx_slider = widgets.IntSlider(min=0, max=len(images)-1, step=1, value=0)
    
    warp_toggle = widgets.widget_bool.ToggleButton(value=True)

    # Sobel kernel size - odd number - larger = smoother gradient measurements - max 31
    grad_ksize_slider = widgets.IntSlider(min=3, max=31, step=2, value=27) #3
    mag_ksize_slider = widgets.IntSlider(min=3, max=31, step=2, value=27)  #3
    dir_ksize_slider = widgets.IntSlider(min=3, max=31, step=2, value=15)  #3

    gradx_min_slider = widgets.IntSlider(min=0, max=255, step=1, value=30)  #20
    gradx_max_slider = widgets.IntSlider(min=0, max=255, step=1, value=120)  #80

    grady_min_slider = widgets.IntSlider(min=0, max=255, step=1, value=30)  #30
    grady_max_slider = widgets.IntSlider(min=0, max=255, step=1, value=120)  #80

    mag_min_slider = widgets.IntSlider(min=0, max=255, step=1, value=25)  #15
    mag_max_slider = widgets.IntSlider(min=0, max=255, step=1, value=120)  #175
    
    dir_min_slider = widgets.FloatSlider(min=0, max=np.pi/2, step=0.05, value=0.7) #0.6
    dir_max_slider = widgets.FloatSlider(min=0, max=np.pi/2, step=0.05, value=1.57) #1.2

    sat_min_slider = widgets.IntSlider(min=0, max=255, step=1, value=120) #140
    sat_max_slider = widgets.IntSlider(min=0, max=255, step=1, value=200) #255

    lum_min_slider = widgets.IntSlider(min=0, max=255, step=1, value=160) #198
    lum_max_slider = widgets.IntSlider(min=0, max=255, step=1, value=255) #255

    lumsat_min_slider = widgets.IntSlider(min=0, max=255, step=1, value=145)
    lumsat_max_slider = widgets.IntSlider(min=0, max=255, step=1, value=255)
    
    w=widgets.interactive(update_demo_thorough, imgs=images, do_warp=warp_toggle,
                          grad_ksize=grad_ksize_slider, mag_ksize=mag_ksize_slider, dir_ksize=dir_ksize_slider,
                          gradx_min=gradx_min_slider, gradx_max=gradx_max_slider, 
                          grady_min=grady_min_slider, grady_max=grady_max_slider,
                          mag_min=mag_min_slider, mag_max=mag_max_slider,
                          dir_min=dir_min_slider, dir_max=dir_max_slider,
                          sat_min=sat_min_slider, sat_max=sat_max_slider,
                          lum_min=lum_min_slider, lum_max=lum_max_slider,
                          lumsat_min=lumsat_min_slider, lumsat_max=lumsat_max_slider)
    display(w)

# UNCOMMENT TO RUN
demonstrate_filter_parameters(test_images)
#demonstrate_filter_parameters(test_images + pv_images)
#demonstrate_filter_parameters(pv_images)

In [ ]:
# 4-8 INCOMPLETE EXPERIMENT
def demonstrate_adding_img_channels(fname):
    
    img = mpimg.imread(fname)
    img = cal_undistort(img)
    
    #img = birdseye.warp(img)
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)  
    h1, s1, v1 = cv2.split(hsv)
    
    #st = 180
    #vt = 198
    #s1[s1 < st] = 0
    #s1[s1 > st] = 255
    #v1[v1 < vt] = 0
    #v1[v1 > vt] = 255
    sv = (np.float32(s1) + np.float32(v1))//2
    sv[sv < 90] = 0
    svrgb = cv2.cvtColor(np.uint8(sv), cv2.COLOR_GRAY2RGB)
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h2, l2, s2 = cv2.split(hls)
    ls = (np.float32(l2) + np.float32(s2))//2
    ls[ls < 95] = 0
    lsrgb = cv2.cvtColor(np.uint8(ls), cv2.COLOR_GRAY2RGB)
    
    svls = (np.float32(l2) + np.float32(s2))//2
    svls[svls < 128] = 0
    svls4 = (np.float32(s1) + np.float32(v1) + np.float32(l2) + np.float32(s2))//4
    svls4[svls4 < 133] = 0
    svls4[svls4 > 132] = 255
    
    svlsrgb = cv2.cvtColor(np.uint8(svls), cv2.COLOR_GRAY2RGB)
    svls4rgb = cv2.cvtColor(np.uint8(svls4), cv2.COLOR_GRAY2RGB)
    
    ksize=3
    gradx_thresh=(20, 80)
    grady_thresh=(30, 170)
    mag_thresh=(15, 175)
    dir_thresh=(0.6, 1.2)
    #sat_thresh=(140, 255)
    #lum_thresh=(198, 255)
            
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(s1, orient='x', sobel_kernel=ksize, thresh=gradx_thresh)
    grady = abs_sobel_thresh(s1, orient='y', sobel_kernel=ksize, thresh=grady_thresh)
    mag_binary = grad_magnitude_thresh(s1, sobel_kernel=ksize, thresh=mag_thresh)
    dir_binary = grad_direction_thresh(s1, sobel_kernel=ksize, thresh=dir_thresh)
    
    grad = (np.float32(gradx) + np.float32(grady))//2
    magdir = (np.float32(mag_binary) + np.float32(dir_binary))//2
    
    #biggie = (np.float32(s1) + np.float32(v1) + np.float32(l2) + np.float32(s2) + 
    #          np.float32(gradx) + np.float32(grady) + np.float32(mag_binary) + np.float32(dir_binary))//8
    #biggie[biggie < 70] = 0
    #biggie[biggie > 69] = 255
    biggie = (np.float32(sv) + np.float32(ls) + 
              np.float32(gradx) + np.float32(grady) + np.float32(mag_binary) + np.float32(dir_binary))//6
    biggie[biggie < 40] = 0
    biggie[biggie > 39] = 255

    biggiergb = cv2.cvtColor(np.uint8(biggie), cv2.COLOR_GRAY2RGB)

    f, axes = plt.subplots(5,3, figsize=(20,20))
    
    axes[0][0].imshow(s1, cmap='gray')
    axes[0][1].imshow(v1, cmap='gray')
    axes[0][2].imshow(svrgb)

    axes[1][0].imshow(l2, cmap='gray')
    axes[1][1].imshow(s2, cmap='gray')
    axes[1][2].imshow(lsrgb)
    
    axes[2][0].imshow(img)
    axes[2][1].imshow(svls4rgb)
    axes[2][2].imshow(biggiergb)

    axes[3][0].imshow(gradx, cmap='gray')
    axes[3][1].imshow(grady, cmap='gray')
    axes[3][2].imshow(grad, cmap='gray')

    axes[4][0].imshow(mag_binary, cmap='gray')
    axes[4][1].imshow(dir_binary, cmap='gray')
    axes[4][2].imshow(magdir, cmap='gray')

    plt.show()

# UNCOMMENT TO RUN
#demonstrate_adding_img_channels(test_images[7])

In [ ]:
# 4-9 INCOMPLETE EXPERIMENT
def demonstrate_adding_img_channels_alt(fname, warp_first=True):
    
    img = mpimg.imread(fname)
    img = cal_undistort(img)
    
    if (warp_first):
        img = birdseye.warp(img)
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)  
    h1, s1, v1 = cv2.split(hsv)

    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h2, l2, s2 = cv2.split(hls)

    s1o = np.copy(s1)
    s1t = 127
    s1[s1<s1t] = 0
    s1[s1>s1t-1] = 255
    
    s2o = np.copy(s2)
    s2t = 100
    s2[s2<s2t] = 0
    s2[s2>s2t-1] = 255

    v1o = np.copy(v1)
    v1t = 200
    v1[v1<v1t] = 0
    v1[v1>v1t-1] = 255    

    l2o = np.copy(l2)
    l2t = 200
    l2[l2<l2t] = 0
    l2[l2>l2t-1] = 255

    sv = (np.float32(s1o) + np.float32(v1o))//2
    svo = np.copy(sv)
    svt = 135
    sv[sv < svt] = 0
    sv[sv > svt-1] = 255
    svo_rgb = cv2.cvtColor(np.uint8(svo), cv2.COLOR_GRAY2RGB)
    sv_rgb = cv2.cvtColor(np.uint8(sv), cv2.COLOR_GRAY2RGB)

    ls = (np.float32(l2o) + np.float32(s2o))//2
    lso = np.copy(ls)
    lst = 125
    ls[ls < lst] = 0
    ls[ls > lst-1] = 255
    lso_rgb = cv2.cvtColor(np.uint8(lso), cv2.COLOR_GRAY2RGB)
    ls_rgb = cv2.cvtColor(np.uint8(ls), cv2.COLOR_GRAY2RGB)

    comb = (np.float32(s1) + np.float32(v1) + np.float32(l2) + np.float32(s2) + np.float32(sv) + np.float32(ls))//6
    combt = 127
    comb[comb<combt] = 0
    comb[comb>combt-1] = 255
    comb_rgb = cv2.cvtColor(np.uint8(comb), cv2.COLOR_GRAY2RGB)
    
    if (not warp_first):
        comb_rgb = birdseye.warp(comb_rgb)
     
    f, axes = plt.subplots(7,2, figsize=(20,40))
    
    axes[0][0].imshow(img)
    axes[0][0].set_title("Original")
    axes[0][1].imshow(comb_rgb)
    axes[0][1].set_title("Combined")

    axes[1][0].imshow(s1o, cmap='gray')
    axes[1][0].set_title("HSV Sat")
    axes[1][1].imshow(s1, cmap='gray')
    axes[1][1].set_title("HSV Sat Thresh")

    axes[2][0].imshow(s2o, cmap='gray')
    axes[2][0].set_title("HLS Sat")
    axes[2][1].imshow(s2, cmap='gray')
    axes[2][1].set_title("HLS Sat Thresh")
    
    axes[3][0].imshow(v1o, cmap='gray')
    axes[3][0].set_title("HSV Val")
    axes[3][1].imshow(v1, cmap='gray')
    axes[3][1].set_title("HSV Val Thresh")

    axes[4][0].imshow(l2o, cmap='gray')
    axes[4][0].set_title("HLS Lum")
    axes[4][1].imshow(l2, cmap='gray')
    axes[4][1].set_title("HLS Lum Thresh")

    axes[5][0].imshow(svo_rgb)
    axes[5][0].set_title("HSV Sat+Val")
    axes[5][1].imshow(sv_rgb)
    axes[5][1].set_title("HSV Sat+Val Thresh")

    axes[6][0].imshow(lso_rgb)
    axes[6][0].set_title("HLS Lum+Sat")
    axes[6][1].imshow(ls_rgb)
    axes[6][1].set_title("HLS Lum+Sat Thresh")

    plt.show()

# UNCOMMENT TO RUN
#demonstrate_adding_img_channels_alt(test_images[6])

In [ ]:
# 4-10 Visualize the birdseye transform on test images

# Performing the warp first and then threshold results in better combined output
# than threshold first then warp.

def demonstrate_birdseye_transform_with_thresholds(imgs):

    for fname in imgs:

        img = mpimg.imread(fname)
        undistorted = cal_undistort(img)
        undistorted_warped = birdseye.warp(undistorted)

        binary = combined_thresh(undistorted)
        binary_warped = birdseye.warp(binary)

        undistorted_warped_binary = combined_thresh(undistorted_warped)

        f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(20, 22))
        f.tight_layout()

        ax1.imshow(undistorted)
        ax1.set_title('Undistorted', fontsize=30)
        ax1.add_patch(Polygon(birdseye.src, True, edgecolor='#ff0000', fill=False))

        ax2.imshow(undistorted_warped)
        ax2.set_title('Undistorted -> Warped', fontsize=30)
        ax2.add_patch(Polygon(birdseye.dst, True, edgecolor='#ff0000', fill=False))

        ax3.imshow(binary, cmap='gray')
        ax3.set_title('Undistorted -> Thresholded', fontsize=30)
        ax3.add_patch(Polygon(birdseye.src, True, edgecolor='#ff0000', fill=False))

        ax4.imshow(binary_warped, cmap='gray')
        ax4.set_title('Thresholded -> Warped', fontsize=30)
        ax4.add_patch(Polygon(birdseye.dst, True, edgecolor='#ff0000', fill=False))

        f.delaxes(ax5)

        ax6.imshow(undistorted_warped_binary, cmap='gray')
        ax6.set_title('Undistorted -> Warped -> Thresholded', fontsize=30)
        ax6.add_patch(Polygon(birdseye.dst, True, edgecolor='#ff0000', fill=False))

        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        plt.show()

# UNCOMMENT TO RUN
demonstrate_birdseye_transform_with_thresholds([test_images[0], test_images[1]])

In [ ]:
# 4-11 Define a convenience method for performing the undistort, birdseye warp, 
#      and threshold functions all in one fell (fowl?) swoop.

def get_birdseye_binary_warped(img, undistort=True):
    ''' Convenience method.
    Undistorts an image (using previously determined globally accessible
    calibration data), warps it to the birdseye view, converting it to a uint8 
    after warping, then applying the combined threshold.
    Optionally: skip the undistort step.
    '''
    global birdseye, objpoints, imgpoints
    
    if (undistort):
        undistorted = cal_undistort(img)
    else:
        undistorted = img
    
    # Apply the thresholds
    #binary = combined_thresh(undistorted)
    # Warp to birds-eye view
    #return birdseye.warp(binary)

    # Warp to birds-eye view
    masked = birdseye.apply_cropping_mask(undistorted)
    warped = birdseye.warp(masked)
    
    # Apply the thresholds
    return davg_thresh(warped)

In [ ]:
# 4-12 Demonstrate the direction gradient threshold on an image versus
#      the birdseye warped version of the image.

def demonstrate_direction_gradient_comparison(fname, dir_ksize=15, dir_min=0.7, dir_max=np.pi/2):
    
    img = mpimg.imread(fname)
    img = cal_undistort(img)
    
    # Grayscale
    gry = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gry_rgb = cv2.cvtColor(gry, cv2.COLOR_GRAY2RGB)

    # Dir of Grayscale
    dir_binary = grad_direction_thresh(gry, sobel_kernel=dir_ksize, thresh=(dir_min, dir_max))
    
    # Inverted Dir of Grayscale
    inv_dir_binary = np.zeros_like(dir_binary)
    inv_dir_binary[dir_binary == 0] = 1
    
    # Warped Grayscale
    warped = birdseye.warp(img)
    warped_gry = cv2.cvtColor(warped, cv2.COLOR_RGB2GRAY)
    warped_gry_rgb = cv2.cvtColor(warped_gry, cv2.COLOR_GRAY2RGB)
    
    # Dir of Warped Grayscale
    warped_dir_binary = grad_direction_thresh(warped_gry, sobel_kernel=dir_ksize, thresh=(dir_min, dir_max))
    
    # Inverted Dir of Warped Grayscale
    inv_warped_dir_binary = np.zeros_like(warped_dir_binary)
    inv_warped_dir_binary[warped_dir_binary == 0] = 1    
    
    screen = compose_2x3_screen(diag1=gry_rgb, diag2=dir_binary, diag3=inv_dir_binary, 
                       diag4=warped_gry_rgb, diag5=warped_dir_binary, diag6=inv_warped_dir_binary, 
                       title1="Grayscale", title2="Direction", title3="Inverted Direction", 
                       title4="Warped Grayscale", title5="Warped Direction", title6="Inverted Warped Direction", 
                       color=(255,0,0), thickness=5)
    plt.figure(figsize=(12,12))
    plt.imshow(screen)
    plt.axis('off')
    plt.show()
    
# UNCOMMENT TO RUN
#demonstrate_direction_gradient_comparison(test_images[0])

## GOAL 5: Detect lane pixels and fit to find the lane boundary.
## GOAL 6: Determine the curvature of the lane and vehicle position with respect to center.

In [ ]:
# 5-1 Visualize the birdseye warp on test images
#     Also: demonstrate taking a histogram of the lower half of the
#     image for determining starting points for finding lane lines

def demonstrate_birdseye_warp_with_histogram(imgs):
    for fname in imgs:

        # Read in a test image
        img = mpimg.imread(fname)

        # Undistort, threshold, warp
        binary_warped = get_birdseye_binary_warped(img)
        
        print ("binary_warped.shape {}".format(binary_warped.shape))
        
        # Count up occurances of 1-values of pixels for lower half of image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

        f, (ax1, ax2) = plt.subplots(1,2)
        f.tight_layout()

        ax1.imshow(binary_warped, cmap='gray')
        ax1.set_title("Birdseye Threshold")
        
        # highlight region that is being considered for histogram
        """
        ax1.add_patch(Polygon([(0, binary_warped.shape[0]), 
                               (0, binary_warped.shape[0]//2),
                               (binary_warped.shape[1]-1, binary_warped.shape[0]//2),
                               (binary_warped.shape[1]-1, binary_warped.shape[0])], True, alpha=0.4, color='#eeeeee'))
        """
        ax2.plot(histogram, color='#ff0000')
        ax2.set_title("Histogram")

        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        plt.show()
    
# UNCOMMENT TO RUN
demonstrate_birdseye_warp_with_histogram([test_images[0]])

In [ ]:
# 5-2 Define functions for calculating the curvature of the lane lines
#     in terms of the radius of a circle tangent to an evaluation point.

def get_curve_radius(eval_value, fit_coef):
    '''
    Return the radius of the curve at the specified point.
    Using f(y) = Ay^2 + By + C
    Where y = eval_value and [A,B,C] are the fit_coef values
    Radius = [1 + (dx/dy)^2]^3/2 / |d^2x / dy^2|
           = (1 + (2Ay + B)^2)^3/2 / |2A|
    http://www.intmath.com/applications-differentiation/8-radius-curvature.php
    https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/2b62a1c3-e151-4a0e-b6b6-e424fa46ceab/lessons/40ec78ee-fb7c-4b53-94a8-028c5c60b858/concepts/2f928913-21f6-4611-9055-01744acc344f    
    '''
    return ((1 + (2*fit_coef[0]*eval_value + fit_coef[1])**2)**1.5) / np.absolute(2*fit_coef[0])

def convert_x_pixels_to_meters(values):
    return np.multiply(values, 0.00528571) # 3.7/700 meters/pixel in x dimension

def convert_y_pixels_to_meters(values):
    return np.multiply(values, 0.04166667) # 30/720 meters/pixel in y dimension

def convert_pixels_to_meters(x_values, y_values):
    ''' Convenience method. Scales x and y coordinates from the birds-eye view to their
        equivalent values in meters in real-world space.
    ''' 
    return convert_x_pixels_to_meters(x_values), convert_y_pixels_to_meters(y_values)

def get_curve_radius_in_meters(ploty, x_values):
    ''' Scales plot points for a birds-eye curve from pixel coordinates to
        meters, re-performs a polyfit to get the appropriate polynomial
        coefficients, then evaluates the curve radius equation at the
        point at the bottom of the image.
    '''
    
    # Convert the pixel values to meter values
    conv_x, conv_y = convert_pixels_to_meters(x_values, ploty)
    
    # Find new fit polynomial values based on the new input
    conv_fit = np.polyfit(conv_y, conv_x, 2)
    
    # Grab the max y-value
    y_eval = np.max(conv_y)
    
    # Return the radius at that point
    return get_curve_radius(y_eval, conv_fit)

def get_curve_radii_in_pixels(ploty, left_fit, right_fit):
    ''' Convenience method for getting the curvature of the left and right
        lane lines in pixels.
    '''
    
    # Define y-value where we want radius of curvature
    # In this case, the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    left_curverad = get_curve_radius(y_eval, left_fit)
    right_curverad = get_curve_radius(y_eval, right_fit)
    
    return left_curverad, right_curverad

def get_curve_radii_in_meters(ploty, leftx, rightx):
    ''' Convenience method for getting the curvature of the left and right
        lane lines in meters.
    '''
    left_curverad = get_curve_radius_in_meters(ploty, leftx)
    right_curverad = get_curve_radius_in_meters(ploty, rightx)
    
    return left_curverad, right_curverad

In [ ]:
# 5-3 Define functions for calculating the offset from center

def eval_poly_at(at, poly_coefficients):
    ''' Creates the polynomial defined by the coefficients, then evaluates it at the specified value(s).
        If 'at' is a scalar, returns a scalar. If it is an array, performs it for all values and returns
        an array of the same dimensions.
    '''
    poly = np.poly1d(poly_coefficients)
    return poly(at)    

def get_lane_center_in_pixels(ploty, left_fit, right_fit):
    ''' Find the center pixel value between the right and left lines at the bottom of the image.
    '''
    # Grab the y point at the bottom of the image
    y_eval = np.max(ploty)
    
    # Evaluate the x points at that y-point
    left_x = eval_poly_at(y_eval, left_fit)
    right_x = eval_poly_at(y_eval, right_fit)
    
    return ((right_x + left_x) // 2)

def get_lane_offset_in_meters(img_width, lane_center):
    ''' Converts the pixel offset to meters '''
    return convert_x_pixels_to_meters(img_width//2 - lane_center)

In [ ]:
# 5-4 Define functions for finding lane lines using the sliding windows method
#     described in the Udacity 'Finding the Lines' lesson

def find_lane_lines_using_windows(binary_warped):

    # This code was taken from the Udacity 'Finding the Lines' section of the
    # Advanced Lane Finding lesson
    # https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/2b62a1c3-e151-4a0e-b6b6-e424fa46ceab/lessons/40ec78ee-fb7c-4b53-94a8-028c5c60b858/concepts/c41a4b6b-9e57-44e6-9df9-7e4e74a1a49a
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Set the width of the windows +/- margin
    margin = 120
    
    # Set minimum number of pixels found to recenter window
    minpix = 50
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height  # bottom of image - (next window count * window height)
        win_y_high = binary_warped.shape[0] - window*window_height     # bottom of image - (curr window count * window height)
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Color the non-zero values that are part of the lanes
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit, out_img

def visualize_lanes_using_windows(img):
    
    # Undistort, threshold, warp
    binary_warped = get_birdseye_binary_warped(img)

    left_fit, right_fit, out_img = find_lane_lines_using_windows(binary_warped)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = eval_poly_at(ploty, left_fit)
    right_fitx = eval_poly_at(ploty, right_fit)
    
    # Visualize the lines
    plt.figure()
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1296)
    plt.ylim(994, 0)
    plt.show()
    
    left_curverad_px, right_curverad_px = get_curve_radii_in_pixels(ploty, left_fit, right_fit)
    print("left curve {:.3f} px, right curve {:.3f} px".format(left_curverad_px, right_curverad_px))

    left_curverad_m, right_curverad_m = get_curve_radii_in_meters(ploty, left_fitx, right_fitx)
    print("left curve {:.3f} m, right curve {:.3f} m".format(left_curverad_m, right_curverad_m))
    
    lane_center_px = get_lane_center_in_pixels(ploty, left_fit, right_fit)
    lane_offset_m = get_lane_offset_in_meters(binary_warped.shape[1], lane_center_px)
    print("lane center {:.3f} px, offset from lane center {:.3f} m".format(lane_center_px, lane_offset_m))
    
    return left_fit, right_fit

In [ ]:
# 5-5 Visualize the sliding windows functionality on test images

def demonstrate_sliding_windows(data):
    for idx in range(len(data)):

        # Read in a test image
        img = mpimg.imread(data[idx])
        
        print ("img.shape", img.shape)

        # Process it
        left_fit, right_fit = visualize_lanes_using_windows(img)

# UNCOMMENT TO RUN
demonstrate_sliding_windows(test_images)

In [ ]:
# 5-6 Define functions for finding lane lines using previously found fit lines as
#     described in the Udacity 'Finding the Lines' lesson

def find_lane_lines_from_fit(binary_warped, left_fit, right_fit):

    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Set the width of the windows +/- margin
    margin = 100

    # Determine the location of the left and right lane indices
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Color in left and right line pixels
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit, out_img

def visualize_lines_from_fit(img, l_fit, r_fit):
    
    # Undistort, threshold, warp
    binary_warped = get_birdseye_binary_warped(img)

    left_fit, right_fit, out_img = find_lane_lines_from_fit(binary_warped, l_fit, r_fit)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = eval_poly_at(ploty, left_fit)
    right_fitx = eval_poly_at(ploty, right_fit)

    # Set the width of the windows +/- margin
    margin = 100
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Create an image to show the selection window
    window_img = np.zeros_like(out_img)
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))

    # Draw the selection window onto the output image
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    plt.figure()
    plt.imshow(result)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1296)
    plt.ylim(994, 0)
    plt.show()
    
    left_curverad_px, right_curverad_px = get_curve_radii_in_pixels(ploty, left_fit, right_fit)
    print("left curve {:.3f} px, right curve {:.3f} px".format(left_curverad_px, right_curverad_px))

    left_curverad_m, right_curverad_m = get_curve_radii_in_meters(ploty, left_fitx, right_fitx)
    print("left curve {:.3f} m, right curve {:.3f} m".format(left_curverad_m, right_curverad_m))
    
    lane_center_px = get_lane_center_in_pixels(ploty, left_fit, right_fit)
    lane_offset_m = get_lane_offset_in_meters(binary_warped.shape[1], lane_center_px)
    print("lane center {:.3f} px, offset from lane center {:.3f} m".format(lane_center_px, lane_offset_m))

    return left_fit, right_fit

In [ ]:
# 5-7 Now run our visualization functions defined above on some of the images

def demonstrate_image_processing_on_video_frames():
    left_fit = []
    right_fit = []

    for img in extract_frames('project_video', start_time=0.0, interval=0.1, max_images=3):

        '''
        # Uncomment this code to further visualize some of the intermediate states
        # of the image processing pipeline.
        
        undistorted = cal_undistort(img)
        undistorted_warped = birdseye.warp(undistorted)

        binary = combined_thresh(undistorted)
        binary_warped = birdseye.warp(binary)


        f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 14))
        f.tight_layout()

        ax1.imshow(undistorted)
        ax1.set_title('Undistorted', fontsize=50)
        ax1.add_patch(Polygon(birdseye.src, True, edgecolor='#ff0000', fill=False))

        ax2.imshow(undistorted_warped)
        ax2.set_title('Undistorted -> Warped', fontsize=50)
        ax2.add_patch(Polygon(birdseye.dst, True, edgecolor='#ff0000', fill=False))

        ax3.imshow(binary, cmap='gray')
        ax3.set_title('Undistorted -> Thresholded', fontsize=50)
        ax3.add_patch(Polygon(birdseye.src, True, edgecolor='#ff0000', fill=False))

        ax4.imshow(binary_warped, cmap='gray')
        ax4.set_title('Thresholded -> Warped', fontsize=50)
        ax4.add_patch(Polygon(birdseye.dst, True, edgecolor='#ff0000', fill=False))

        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        plt.show()
        '''

        if ((len(left_fit) == 0) or (len(right_fit) == 0)):
            left_fit, right_fit = visualize_lanes_using_windows(img)
        else:
            left_fit, right_fit = visualize_lines_from_fit(img, left_fit, right_fit)
            
# UNCOMMENT TO RUN
#demonstrate_image_processing_on_video_frames()

## GOAL 7: Warp the detected lane boundaries back onto the original image.
## GOAL 8: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

Note: the line-finding code above has been re-implemented below using a Line class to maintain state between frames and logic has been put in place to check the sanity of the fit curves.

In [ ]:
# 7-1 Define our own line plotting function using numpy and opencv
#     so that plotting to a matplotlib figure doesn't need to be used.

def plot_line(img, x, y, color=(255,255,0), thickness=2):
    ''' Takes an image and two arrays of x and y points similar to matplotlib
        and writes the lines onto the image. If the points are floats, they
        are rounded and converted to ints to satisfy opencv.
    '''
    points = np.rint(np.vstack([x,y]).T).astype(int)
    #print(points)
    cv2.polylines(img, [points], False, color, thickness)

In [ ]:
# 7-2 Test our line plotting function

def demonstrate_plotting_function():
    test_img = np.zeros((128, 128, 3), dtype='uint8')

    x = np.array([40.1,60,80,100])
    y = np.array([82.6,102,50,20])

    plot_line(test_img, x, y)

    plt.imshow(test_img)
    plt.show()
    
# UNCOMMENT TO RUN
#demonstrate_plotting_function()

In [ ]:
# 7-3 Define some functions for helping to predict where the lane might be
#     based on previous line data in the event that a line can't be determined.

# Predicts the next values based on a softmax weighted averages of the
# differences between the n-last previous values.

def softmax(x):
    '''Compute softmax values for each value in x.'''
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def find_weighted_averages(data, window=2):
    ''' Given an array of arrays, calculates the averages along the 0 axis for
        for the past few elements (default 2) weighted by a softmax function,
        with the heaviest weights at the end of the window.
        'window' must be an integer between 1 and the length of the enclosing array.
        Returns a numpy array.
    '''
    result = []
    weights = softmax(np.array(list(range(window))))

    for i in range(len(data)):
    
        if (i >= window-1):
            # Use the full window previously defined if possible
            avg = np.average(data[i-(window-1):i+1], axis=0, weights=weights)
        else:
            # Otherwise, too close to an edge so recalculate weights for smaller window
            alt_weights = softmax(np.array(list(range(i+1))))
            avg = np.average(data[0:i+1], axis=0, weights=alt_weights)

        result.append(avg)

    return result

def predict_next_values(data, window=2):
    ''' Predict the next set of numbers by applying the last weighted avg of the diffs to the last data set '''

    # If empty array, just return it
    if (len(data) == 0):
        return data

    # If there's only one element, return that element as the prediction    
    if (len(data) == 1):
        return data[0]

    # Otherwise perform the weighted average of the diffs
    diffs = np.diff(data, axis=0)
    wavgs = find_weighted_averages(diffs, window=window)
    return np.add(wavgs[-1], data[-1])

In [ ]:
# 7-4 Test performing a softmax weighted average over a series of points 
#     and predicting next set of points.

def demonstrate_weighted_average_and_prediction():

    # Create a blank array to be used as an image
    test_img = np.zeros((128, 128, 3), dtype='uint8')

    # Define common y-points
    y = np.array([0,31,63,95,127])

    # Define an array of x-point arrays
    #recent_x = np.array([[40,40,40,40,40]])
    #recent_x = np.array([[40,40,40,40,40], [30,35,37,39,40]])
    #recent_x = np.array([[40,40,40,40,40], [30,35,37,39,40], [20,30,35,38,40], [10,25,32,37,40]])
    #recent_x = np.array([[40,40,40,40,40], [30,35,37,39,40], [20,30,35,38,40], [10,25,32,37,40], [20,30,35,38,40]])
    recent_x = np.array([[40,40,40,40,40], [30,35,37,39,40], [20,30,35,38,40], [10,25,32,37,40], [0,20,29,36,40]])
    print ("recent_x", recent_x)

    # Calculate the softmax weighted averages for the x-points
    averages = find_weighted_averages(recent_x, window=3)
    print("weighted averages", averages)

    # Calculate the differences between the each consecutive set of x-points
    recent_xdiff = np.diff(recent_x, axis=0)
    print ("recent_xdiff", recent_xdiff)

    if len(recent_xdiff) != 0:
        # Calculate the non-weighted average of the differences for a baseline
        recent_xdiff_avg = np.average(recent_xdiff, axis=0)
        print ("recent_xdiff_avg", recent_xdiff_avg)

        # Calculate the softmax weighted averages for the differences in the x-points
        xdiff_weighted_averages = find_weighted_averages(recent_xdiff, window=2)
        print("xdiff_weighted_averages[-1]:", xdiff_weighted_averages[-1])

    # Predict the next line location by applying the last weighted diff to the last x-points 
    #predicted_x = np.add(xdiff_weighted_averages[-1], recent_x[-1])
    predicted_x = predict_next_values(recent_x, window=2)
    print("predicted:", predicted_x)

    # Plot the various lines
    for i in range(len(recent_x)):
        # Plot a red line for the weighted moving averages
        plot_line(test_img, averages[i], y, thickness=1, color=(200,0,0))

        # Plot a yellow line for the current points
        plot_line(test_img, recent_x[i], y, thickness=1)

    # Plot a green line for the predicted next line based on weighted averages of the diffs
    plot_line(test_img, predicted_x, y, thickness=1, color=(0,200,0))

    plt.imshow(test_img)
    plt.show()

# UNCOMMENT TO RUN
#demonstrate_weighted_average_and_prediction()

In [ ]:
# 7-5 Define a class to receive the characteristics of each line detection

class Line():
    def __init__(self):
        
        # was the line detected in the last iteration?
        self.detected = False
        
        # x-values for detected line pixels
        self.detected_pixelsx = None
        
        # y-values for detected line pixels
        self.detected_pixelsy = None

        # polynomial coefficients for the fit to the detected pixels
        self.detected_fit = [np.array([False])]
        
        # x-values resulting from evaluating detected_fit at the y-values
        self.detected_fitx = None

        # polynomial coefficients for the fit that was actually used
        # This may be the same as detected_fit, or a predicted value based on
        # the differences in the recent history
        self.used_fit = [np.array([False])]
        
        # x-values resulting from evaluating used_fit at the y-values
        self.used_fitx = None        
        
        # Depth of the history to keep
        self.history_depth = 5

        # x-values of the last history_depth fits of the line
        self.recent_fitxs = []
        
        # Polynomial coefficients for the polynomial fit to the best_fitx
        self.best_fit = [np.array([False])]

        # Weighted average of x-values from recent_fitxs values
        self.best_fitx = None
        
        # Image showing detected line pixels decayed over last history_depth iterations
        #self.history_heatmap = None
        
        #radius of curvature of the best_fit line in pixels
        self.radius_of_curvature_px = None
        
        #radius of curvature of the best_fit line in meters
        self.radius_of_curvature_m = None
    
    def predict_next_fitx(self):
        #return self.recent_fitxs[-1]
        return predict_next_values(self.recent_fitxs, window=self.history_depth)

In [ ]:
# 7-6 Demonstrate a gaussian plot

def show_gauss_plot(width=111):
    mu, sigma = 0, 0.2
    bins = np.linspace(-0.6, 0.6, width)
    gauss = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2))
    norm_gauss = gauss / np.max(gauss)
    
    plt.plot(norm_gauss, linewidth=2, color='r')
    plt.show()
    
# UNCOMMENT TO RUN
#show_gauss_plot()

In [ ]:
# 7-7 Calculate a gaussian distribution of the specified width using
#     the pre-defined mu and sigma values below.

def get_gaussian_filter(width=51):
    mu, sigma = 0, 0.2
    bins = np.linspace(-0.6, 0.6, width)
    gauss = 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2))
    return gauss / np.max(gauss)

In [ ]:
# 7-8 Apply a gaussian filter horizontally along a series of x-values to
#     smoothly diminish the effect of pixels to the left and right of the
#     curve.

def apply_horizontal_gaussian(data, xfit, margin=120, min_thresh=255):
    ''' Applies a horizontal gaussian filter for the xfit values
        in the data.
    '''
    # ensure we are dealing with ints since they are used as indexes
    rounded_xfit = np.rint(xfit).astype(int)
    
    # generate the common filter once
    gauss_filter = get_gaussian_filter(margin*2)
    
    # for all rows in the image
    for j in range(data.shape[0]):
        
        # determine the range of the window (don't go past borders)
        mini = max(0, rounded_xfit[j] - margin)
        maxi = min(rounded_xfit[j] + margin, data.shape[1])

        # check the length of the subset we want to modify
        sublen = len(data[j][mini:maxi])
        
        # apply the filter to the subset and put it back into the data
        if (sublen != margin*2):
            # if the size of the subset is less than the intended window, use a custom size filter
            data[j][mini:maxi] = np.multiply(data[j][mini:maxi], get_gaussian_filter(sublen))
        else:
            data[j][mini:maxi] = np.multiply(data[j][mini:maxi], gauss_filter)
        #print("mini:", mini, "maxi:", maxi, "len data[j][mini:maxi]", len(data[j][mini:maxi]), "sublen:", sublen, "margin*2", margin*2, "len gfilt", len(gauss_filter))
    
    # if we want to apply a threshold, select only data greater
    # than that (removes negligible values)
    if (min_thresh < 255):
        mask = data > min_thresh
        data = np.multiply(data, mask)
        
    return data

In [ ]:
# 7-9 Define methods for comparing lines

def determine_line_polynomial_similarity(line1_fit, line2_fit):
    print ("line1 fit:", line1_fit)
    print ("line2 fit:", line2_fit)
    diff = line1_fit - line2_fit
    print ("line fit diff:", diff)
    
def measure_relative_line_curvature(line_fitx):
    len_fitx = len(line_fitx)
    avg_top_fitx = np.mean(line_fitx[:len_fitx//3]).astype(int)
    avg_mid_fitx = np.mean(line_fitx[len_fitx//3:(2*len_fitx)//3]).astype(int)
    avg_bot_fitx = np.mean(line_fitx[(2*len_fitx)//3:]).astype(int)
    
    top_diff = avg_top_fitx - avg_mid_fitx
    bot_diff = avg_mid_fitx - avg_bot_fitx
    
    return top_diff, bot_diff

def lines_are_similar(y, line1_fit, line2_fit):
    ''' TODO: Implement this if necessary '''
    #line_diff = np.subtract(right_fitx, left_fitx).astype(int)
    #line_mean = np.mean(line_diff).astype(int)
    #print("TODO: Implement lines_are_similar (l1:{} l2:{})".format(line1_fit, line2_fit))
    return True

In [ ]:
# 7-10 Define the line finding algorithms using the Line class, do some predictive 
#      assumptions, check sanity of findings, fallback if necessary

def find_line_indices_using_sliding_windows(binary_warped, nonzerox, nonzeroy, margin=110):
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
        
    # Set minimum number of pixels found to recenter window
    minpix = 50
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    left_lane_windows = []
    right_lane_windows = []

    # Step through the windows one by one
    for window in range(nwindows):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height  # bottom of image - (next window count * window height)
        win_y_high = binary_warped.shape[0] - window*window_height     # bottom of image - (curr window count * window height)
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Save the window rect coordinates for drawing later
        left_lane_windows.append([(win_xleft_low,win_y_low),(win_xleft_high,win_y_high)])
        right_lane_windows.append([(win_xright_low,win_y_low),(win_xright_high,win_y_high)])
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    return left_lane_inds, right_lane_inds, left_lane_windows, right_lane_windows

'''
def fit_from_history_heatmap(x, y, ploty, prev_line=None, img=None):
    
    # Grab any historical information we have
    if (prev_line is not None):
        # Decay the previous history heatmap
        history_heatmap = np.rint(prev_line.history_heatmap * 0.8).astype(np.uint8)
        # Grab our next best guess for where the line will be
        next_fitx = prev_line.predict_next_fitx()
    else:
        # If no history heatmap, initialize some
        history_heatmap = np.array(np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8))
        # Generate a fit based on the pixel data we have detected
        next_fit = np.polyfit(y, x, 2)
        # Grab the x-values that fit the line we plan to use
        next_fitx = eval_poly_at(ploty, next_fit)
        
    # Add the new pixel information to the history heatmap
    history_heatmap[y, x] = 255
    
    # Apply a horizontal gausian to focus on where we expect the line to be
    history_heatmap = apply_horizontal_gaussian(history_heatmap, next_fitx, margin=300, min_thresh=143)
    
    # Grab the non-zero coordinates that passed the threshold
    hist_nonzero = history_heatmap.nonzero()
    hist_nz_y = np.array(hist_nonzero[0])
    hist_nz_x = np.array(hist_nonzero[1]) 
    
    # Fit a second order polynomial to
    return np.polyfit(hist_nz_y, hist_nz_x, 2), history_heatmap
'''

def find_lane_lines(binary_warped, margin=100, method='sliding_windows', 
                    prev_left_line=None, prev_right_line=None, produce_out_img=True):

    # This code was adapted from the Udacity 'Finding the Lines' section of the
    # Advanced Lane Finding lesson
    # https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/2b62a1c3-e151-4a0e-b6b6-e424fa46ceab/lessons/40ec78ee-fb7c-4b53-94a8-028c5c60b858/concepts/c41a4b6b-9e57-44e6-9df9-7e4e74a1a49a
    
    ####
    # 1. Try to identify the lane lines in the image
    
    # Identify the x and y positions of all non-zero valued pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Try to select only the points related to the the lines
    
    # If we have a previous line, use the 'previous fit' method to get the indexes
    # of the nonzero values associated with the lines
    if ((method == 'previous_fit') and 
        (prev_left_line is not None) and (prev_left_line.detected != False) and
        (prev_right_line is not None) and (prev_right_line.detected != False)):
        
        # Grab the fitx points along the line for all of the non-zero pixel y-values
        left_nonzerofitx = eval_poly_at(nonzeroy, prev_left_line.best_fit)
        right_nonzerofitx = eval_poly_at(nonzeroy, prev_right_line.best_fit)
        
        # Grab the indices of any non-zero pixels that are within the specified margin of the fitx points
        left_lane_inds = ((nonzerox > (left_nonzerofitx - margin)) & (nonzerox < (left_nonzerofitx + margin))) 
        right_lane_inds = ((nonzerox > (right_nonzerofitx - margin)) & (nonzerox < (right_nonzerofitx + margin)))
        
        # Initialize empty arrays for the windows, which are not used for this method but will
        # be consulted later for drawing to the output image.
        left_lane_windows, right_lane_windows = [], []
    else:
        # Otherwise fall back to the sliding windows method
        left_lane_inds, right_lane_inds, left_lane_windows, right_lane_windows = find_line_indices_using_sliding_windows(binary_warped, nonzerox, nonzeroy, margin=margin)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Generate y values for plotting and fitting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    
    # Fit a second order polynomial using any historical information if possible
    # Otherwise, just use the non-zero pixels we detected
    #left_fit, left_history_heatmap = fit_from_history_heatmap(leftx, lefty, ploty, prev_line=prev_left_line, img=binary_warped)
    #right_fit, right_history_heatmap = fit_from_history_heatmap(rightx, righty, ploty, prev_line=prev_right_line, img=binary_warped)

    # Fit a second order polynomial to each group of pixels
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    if (produce_out_img):
        # Create an output image to draw on and visualize the result and
        # Color the non-zero values that are part of the lanes
        #out_img = np.dstack((left_history_heatmap, np.zeros_like(left_history_heatmap), right_history_heatmap))

        # Create an output image to draw on and visualize the result and
        # Color the non-zero values that are part of the lanes
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        # Draw the windows on the visualization image (if there are any to draw)
        for rect_points in left_lane_windows:
            cv2.rectangle(out_img, rect_points[0], rect_points[1], (0,255,0), 2)
        for rect_points in right_lane_windows:
            cv2.rectangle(out_img, rect_points[0], rect_points[1], (0,255,0), 2)

    ####
    # 2. Now check the sanity of the curves we tried to detect

    sane_left = True
    sane_right = True

    # Generate x-values for plotting and conversion to meters
    left_fitx = eval_poly_at(ploty, left_fit)
    right_fitx = eval_poly_at(ploty, right_fit)

    # Calculate the curvature radius in pixels at the bottom of the image
    y_eval = np.max(ploty)
    left_radius_of_curvature_px = get_curve_radius(y_eval, left_fit)
    right_radius_of_curvature_px = get_curve_radius(y_eval, right_fit)

    # Calculate the curvature radius in meters
    # The image size and fitx values need to be specified since the x- and y-
    # values need to be scaled before fitting a line and evaluating at a point.
    left_radius_of_curvature_m = get_curve_radius_in_meters(ploty, left_fitx)
    right_radius_of_curvature_m = get_curve_radius_in_meters(ploty, right_fitx)

    # - Check that Left and Right curvature is not too small (<100)
    if (left_radius_of_curvature_m < 100):
        print("WARNING: left_radius_of_curvature_m < 100:", left_radius_of_curvature_m)
        sane_left = False        

    if (right_radius_of_curvature_m < 100):
        print("WARNING: right_radius_of_curvature_m < 100:", right_radius_of_curvature_m)
        sane_right = False

    # - Check that Left and Right have similar curvature
    left_shape = measure_relative_line_curvature(left_fitx)
    #print("left_shape", left_shape)
    right_shape = measure_relative_line_curvature(right_fitx)
    #print("right_shape", right_shape)
        
    # - Check that Left and Right are separated by approximately the right distance horizontally
    line_diff = np.subtract(right_fitx, left_fitx).astype(int)
    line_mean = np.mean(line_diff).astype(int)

    if (line_mean > 825) or (line_mean < 525):
        print("WARNING: mean line_diff out of range: 525 > {} > 825".format(line_mean))
        sane_left = False
        sane_right = False
    
    # - Check that Left and Right are roughly parallel 
    norm_line_diff = line_diff - line_mean
    #print("norm_line_diff", norm_line_diff)
    max_line_x_diff = np.max(np.abs(norm_line_diff))
    max_line_x_thresh = 140

    if (max_line_x_diff > max_line_x_thresh):
        print("WARNING: max line x diff {} > thresh {}".format(max_line_x_diff, max_line_x_thresh))
        sane_left = False
        sane_right = False

    ####
    # 3. Depending on the sanity check results and past history, figure out what values to use going forward
    
    left_line = Line()
    left_line.detected_fit = left_fit
    left_line.detected_pixelsx = leftx
    left_line.detected_pixelsy = lefty
        
    right_line = Line()
    right_line.detected_fit = right_fit
    right_line.detected_pixelsx = rightx
    right_line.detected_pixelsy = righty

    if (sane_left):
        left_line.detected = True
    else:
        if (prev_left_line is not None):
            left_line.detected = lines_are_similar(ploty, prev_left_line.best_fit, left_fit)
        
    if (sane_right):
        right_line.detected = True    
    else:
        if (prev_right_line is not None):
            right_line.detected = lines_are_similar(ploty, prev_right_line.best_fit, right_fit)
    
    if ((left_line.detected is False) and (prev_left_line is not None)):
        # Predict based on history available in recent_fitxs
        left_line.used_fitx = prev_left_line.predict_next_fitx()
        left_line.used_fit = np.polyfit(ploty, left_line.used_fitx, 2)        
    else:
        # Either the line was detected successfully, so use it, or
        # we don't have any history to use, so we have no choice but use the fit we have. 
        # The sliding windows method will be used next time anyway.       
        left_line.used_fit = left_fit
        left_line.used_fitx = left_fitx

    if ((right_line.detected is False) and (prev_right_line is not None)):
        # Predict based on history available in recent_fitxs
        right_line.used_fitx = prev_right_line.predict_next_fitx()
        right_line.used_fit = np.polyfit(ploty, right_line.used_fitx, 2)        
    else:
        # Either the line was detected successfully, so use it, or
        # we don't have any history to use, so we have no choice but use the fit we have. 
        # The sliding windows method will be used next time anyway.       
        right_line.used_fit = right_fit
        right_line.used_fitx = right_fitx       
    
    ###
    # 4. Update our recent history and best evaluations
    
    # Copy previous recent_fitxs values if available
    if (prev_left_line is not None):
        if (len(prev_left_line.recent_fitxs) == prev_left_line.history_depth):
            left_line.recent_fitxs = prev_left_line.recent_fitxs[1:]
        else:
            left_line.recent_fitxs = prev_left_line.recent_fitxs[:]
    # Append the new used_fitx value to the history
    left_line.recent_fitxs.append(left_line.used_fitx)

    # Copy previous recent_fitxs values if available
    if (prev_right_line is not None):
        if (len(prev_right_line.recent_fitxs) == prev_right_line.history_depth):
            right_line.recent_fitxs = prev_right_line.recent_fitxs[1:]
        else:
            right_line.recent_fitxs = prev_right_line.recent_fitxs[:]    
    # Append the new used_fitx value to the history
    right_line.recent_fitxs.append(right_line.used_fitx)

    # Update the best_fit and best_fitx values
    left_line.best_fitx = find_weighted_averages(left_line.recent_fitxs, left_line.history_depth)[-1]
    left_line.best_fit = np.polyfit(ploty, left_line.best_fitx, 2)
    
    right_line.best_fitx = find_weighted_averages(right_line.recent_fitxs, right_line.history_depth)[-1]
    right_line.best_fit = np.polyfit(ploty, right_line.best_fitx, 2)    

    ####
    # 5. Calculate the radius based on the best values
 
    left_line.radius_of_curvature_px = get_curve_radius(y_eval, left_line.best_fit)
    left_line.radius_of_curvature_m = get_curve_radius_in_meters(ploty, left_line.best_fitx)

    right_line.radius_of_curvature_px = get_curve_radius(y_eval, right_line.best_fit)
    right_line.radius_of_curvature_m = get_curve_radius_in_meters(ploty, right_line.best_fitx)

    if (produce_out_img):
        # Draw the search window on the output image if using the previous fit method
        if (method == 'previous_fit'):

            # Generate a polygon to illustrate the search window area
            # And recast the x and y points into usable format for cv2.fillPoly()
            left_line_window1 = np.array([np.transpose(np.vstack([left_fitx - margin, ploty]))])
            left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx + margin, ploty])))])
            left_line_pts = np.hstack((left_line_window1, left_line_window2))

            right_line_window1 = np.array([np.transpose(np.vstack([right_fitx - margin, ploty]))])
            right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx + margin, ploty])))])
            right_line_pts = np.hstack((right_line_window1, right_line_window2))

            # Create an image to show the selection window
            window_img = np.zeros_like(out_img)
            cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
            cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))

            # Draw the selection window onto the output image
            out_img = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

        # Draw the used lines on the output image
        if (left_line.detected):
            plot_line(out_img, left_line.used_fitx, ploty)
        else:
            plot_line(out_img, left_line.used_fitx, ploty, color=(0,255,255))

        if (right_line.detected):
            plot_line(out_img, right_line.used_fitx, ploty)
        else:
            plot_line(out_img, right_line.used_fitx, ploty, color=(0,255,255))

        # Draw the best lines on the output image
        plot_line(out_img, left_line.best_fitx, ploty, color=(255,0,255))
        plot_line(out_img, right_line.best_fitx, ploty, color=(255,0,255))
    else:
        out_img = None
    
    return left_line, right_line, out_img

In [ ]:
# 7-11 Define some visualization methods

def draw_lane_on_image(img, binary_warped, ploty, left_fitx, right_fitx):
    
    global birdseye
    
    # Create an image to draw the projected lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space
    new_warp = birdseye.unwarp(color_warp)
    
    # Combine the result with the original image
    return cv2.addWeighted(img, 1, new_warp, 0.3, 0)

def visualize_lanes_using_matplotlib(img, left_line=None, right_line=None):
    
    global objpoints, imgpoints
    
    # Undistort, threshold, warp
    img = cal_undistort(img)
    binary_warped = get_birdseye_binary_warped(img, undistort=False)
    
    # Set the width of the windows +/- margin
    margin = 80

    if ((left_line == None) or (right_line == None)):
        left_line, right_line, out_img = find_lane_lines(binary_warped, margin=margin, method='sliding_windows')
    else:
        left_line, right_line, out_img = find_lane_lines(binary_warped, margin=margin, method='previous_fit', 
                                                         prev_left_line=left_line, prev_right_line=right_line)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    result = draw_lane_on_image(img, binary_warped, ploty, left_line.best_fitx, right_line.best_fitx)

    # Visualize the lines
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(out_img)
    ax1.set_title('Birds Eye', fontsize=50)
    ax1.set_xlim(0, 1280)
    ax1.set_ylim(720, 0) 
    ax2.imshow(result)
    ax2.set_title('Lane detected', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

    print("left curve {:.3f} px, right curve {:.3f} px".format(left_line.radius_of_curvature_px, right_line.radius_of_curvature_px))
    print("left curve {:.3f} m, right curve {:.3f} m".format(left_line.radius_of_curvature_m, right_line.radius_of_curvature_m))

    lane_center_px = get_lane_center_in_pixels(ploty, left_line.best_fit, right_line.best_fit)
    lane_offset_m = get_lane_offset_in_meters(binary_warped.shape[1], lane_center_px)
    print("lane center {:.3f} px, offset from lane center {:.3f} m".format(lane_center_px, lane_offset_m))
    
    return left_line, right_line

In [ ]:
# 7-12 Test the new functions on test images

def demonstrate_lane_finding_on_test_images():
    for idx in range(len(test_images)):

        # Read in a test image
        img = mpimg.imread(test_images[idx])

        # Process it
        left_line, right_line = visualize_lanes_using_matplotlib(img)

# UNCOMMENT TO RUN
#demonstrate_lane_finding_on_test_images()

In [ ]:
# 7-13 Test the new functions on frames from the project video

def demonstrate_lane_finding_on_video_frames():
    
    left_line = None
    right_line = None

    i_frame = 0
    for img in extract_frames('project_video', start_time=22.2, interval=1/24, max_images=15):

        print("frame:", i_frame)

        left_line, right_line = visualize_lanes_using_matplotlib(img, left_line, right_line)

        i_frame += 1
        
# UNCOMMENT TO RUN
#demonstrate_lane_finding_on_video_frames()

### Functions to process the video and diagnose the pipeline

In [ ]:
# 7-14 Define some additional diagnostic functions
#      Adapted from https://carnd-forums.udacity.com/questions/32706990/answers/38548228

def compose_diagScreen(curverad=0, offset=0, 
                       mainDiagScreen=None, diag1=None, diag2=None, diag3=None, diag4=None, 
                       diag5=None, diag6=None, diag7=None, diag8=None, diag9=None):

    # Initialize the output image. Dimensions: 1080 H x 1920 W
    diagScreen = np.zeros((1080, 1920, 3), dtype=np.uint8)
    
    # Main screen (720x1280) in upper left
    if mainDiagScreen is not None:
        diagScreen[0:720, 0:1280] = mainDiagScreen
    
    # Four small (240x320) diagnostic screens in upper right
    if diag1 is not None:
        diagScreen[0:240, 1280:1600] = cv2.resize(to_RGB(diag1), (320,240), interpolation=cv2.INTER_AREA) 
    if diag2 is not None:
        diagScreen[0:240, 1600:1920] = cv2.resize(to_RGB(diag2), (320,240), interpolation=cv2.INTER_AREA)
    if diag3 is not None:
        diagScreen[240:480, 1280:1600] = cv2.resize(to_RGB(diag3), (320,240), interpolation=cv2.INTER_AREA)
    if diag4 is not None:
        diagScreen[240:480, 1600:1920] = cv2.resize(to_RGB(diag4), (320,240), interpolation=cv2.INTER_AREA)*4
    
    # Gap of 120x320 on right side
    
    # One medium (480x640) diagnostic screen in lower right
    if diag7 is not None:
        diagScreen[600:1080, 1280:1920] = cv2.resize(to_RGB(diag7), (640,480), interpolation=cv2.INTER_AREA)*4
    
    # Middle panel (120x1280) below main screen on left side

    # Use cv2 for drawing text in diagnostic pipeline.
    font = cv2.FONT_HERSHEY_COMPLEX
    middlepanel = np.zeros((120, 1280, 3), dtype=np.uint8)
    cv2.putText(middlepanel, 'Estimated lane curvature: {:5.3f} m'.format(curverad), (30, 60), font, 1, (255,0,0), 2)
    cv2.putText(middlepanel, 'Estimated offset from center of lane: {:.3f} m'.format(offset), (30, 90), font, 1, (255,0,0), 2)

    diagScreen[720:840, 0:1280] = middlepanel
    
    # Four small (240x320) diagnostic screens in lower left 
    if diag5 is not None:
        diagScreen[840:1080, 0:320] = cv2.resize(to_RGB(diag5), (320,240), interpolation=cv2.INTER_AREA)
    if diag6 is not None:
        diagScreen[840:1080, 320:640] = cv2.resize(to_RGB(diag6), (320,240), interpolation=cv2.INTER_AREA)
    if diag8 is not None:
        diagScreen[840:1080, 640:960] = cv2.resize(to_RGB(diag8), (320,240), interpolation=cv2.INTER_AREA)
    if diag9 is not None:
        diagScreen[840:1080, 960:1280] = cv2.resize(to_RGB(diag9), (320,240), interpolation=cv2.INTER_AREA)

    return diagScreen

In [ ]:
# 7-15 Visualize the various states of the image processing pipeline to diagnose
#      issues.

def visualize_lanes_using_diagnostic_screen(img, left_line=None, right_line=None):
    
    global birdseye, objpoints, imgpoints
    
    # Undistort, threshold, warp
    undistorted = cal_undistort(img)
    
    # Warp to birds-eye view
    warped = birdseye.warp(undistorted)

    # Convert to color spaces
    gry = cv2.cvtColor(warped, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(warped, cv2.COLOR_RGB2HLS)

    grad_ksize=27
    mag_ksize=27
    dir_ksize=15
    gradx_thresh=(30, 120)
    grady_thresh=(30, 120)
    mag_thresh=(30, 120)
    dir_thresh=(0.7, 1.57)
    lumsat_thresh=(145, 255)
            
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gry, orient='x', sobel_kernel=grad_ksize, thresh=gradx_thresh)
    grady = abs_sobel_thresh(gry, orient='y', sobel_kernel=grad_ksize, thresh=grady_thresh)
    mag_binary = grad_magnitude_thresh(gry, sobel_kernel=mag_ksize, thresh=mag_thresh)
    dir_binary = grad_direction_thresh(gry, sobel_kernel=dir_ksize, thresh=dir_thresh)

    lumsat = (np.float32(hls[:,:,1]) + np.float32(hls[:,:,2]))//2
    lumsat_binary = np.zeros_like(gradx)
    lumsat_binary[(lumsat >= lumsat_thresh[0]) & (lumsat <= lumsat_thresh[1])] = 1
    
    # Create combined images for some of these
    blank = np.zeros_like(gry).astype(np.uint8)
    grad = np.dstack((gradx, grady, blank))*255
    magdir = np.dstack((mag_binary, dir_binary, blank))*255
    
    binary_warped = np.zeros_like(dir_binary).astype(np.uint8)
    binary_warped[(((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 0)) | (lumsat_binary == 1))] = 1
        
    # Set the width of the windows +/- margin
    margin = 80

    if ((left_line == None) or (right_line == None)):
        left_line, right_line, out_img = find_lane_lines(binary_warped, margin=margin, method='sliding_windows')
    else:
        left_line, right_line, out_img = find_lane_lines(binary_warped, margin=margin, method='previous_fit', 
                                                         prev_left_line=left_line, prev_right_line=right_line)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    
    undistorted_overlayed = draw_lane_on_image(undistorted, binary_warped, ploty, left_line.best_fitx, right_line.best_fitx)
    
    curverad = (left_line.radius_of_curvature_m + right_line.radius_of_curvature_m) / 2
    
    lane_center_px = get_lane_center_in_pixels(ploty, left_line.best_fit, right_line.best_fit)
    lane_offset_m = get_lane_offset_in_meters(binary_warped.shape[1], lane_center_px)

    screen = compose_diagScreen(curverad=curverad, offset=lane_offset_m, 
                                mainDiagScreen=undistorted_overlayed, 
                                diag1=grad, diag2=magdir, diag3=lumsat_binary, diag4=binary_warped, 
                                diag5=None, diag6=None, diag7=out_img, diag8=None, diag9=None)

    return screen, left_line, right_line

In [ ]:
# 7-16 Test the pipeline on frames from the project video

def demonstrate_lane_finding_on_video_frames_with_diag_screen():
    left_line = None
    right_line = None

    for img in extract_frames('project_video', start_time=41.2, interval=1/24, max_images=3):
        screen, left_line, right_line = visualize_lanes_using_diagnostic_screen(img, left_line, right_line)

        print("left_line.best_fit", left_line.best_fit)
        print("right_line.best_fit", right_line.best_fit)

        plt.figure(figsize=(20,12))
        plt.imshow(screen)
        plt.show()

# UNCOMMENT TO RUN
#demonstrate_lane_finding_on_video_frames_with_diag_screen()

In [ ]:
# 7-17 Process the project video and save the results.

left_line = None
right_line = None

def lane_line_diag(img):
    global left_line, right_line
    screen, left_line, right_line = visualize_lanes_using_diagnostic_screen(img, left_line, right_line)
    return screen

def writeout_lane_finding_video_with_diag_screen(src, dst, start=0, end=0):
    clip = VideoFileClip(src).subclip(start, end)
    diag_clip = clip.fl_image( lane_line_diag )
    diag_clip.write_videofile(dst)

# UNCOMMENT TO RUN
#writeout_lane_finding_video_with_diag_screen('project_video.mp4', 'project_video_test.mp4', start=0, end=None)
#writeout_lane_finding_video_with_diag_screen('challenge_video.mp4', 'challenge_video_test.mp4', start=0, end=None)

### Functions for processing the video in the final form

In [ ]:
# 7-18 Define a function for composing a frame for the final video

def compose_basicScreen(img, curverad=0, offset=0):

    # Determine which side of center in English
    if (offset <= 0):
        side = 'left'
    else:
        side = 'right'
    
    # Make the offset a positive number now that we have the side
    offset = abs(offset)
        
    # Use cv2 for drawing text in diagnostic pipeline.
    font = cv2.FONT_HERSHEY_COMPLEX
    color = (255, 255, 255)
    cv2.putText(img, 'Radius of curvature: {}m'.format(int(curverad)), (30, 50), font, 1, color, 1)
    cv2.putText(img, 'Vehicle is {:.2f}m {} of center'.format(offset, side), (30, 90), font, 1, color, 1)

    return img

# optimization since y dimension doesn't change for our video
g_ploty = np.linspace(0, 719, 720)

def visualize_lane_using_basicScreen(img, left_line=None, right_line=None):
    
    img = cal_undistort(img)
    binary_warped = get_birdseye_binary_warped(img, undistort=False)

    # Set the width of the windows +/- margin
    margin = 80

    if ((left_line == None) or (right_line == None)):
        left_line, right_line, out_img = find_lane_lines(binary_warped, margin=margin, method='sliding_windows', produce_out_img=False)
    else:
        left_line, right_line, out_img = find_lane_lines(binary_warped, margin=margin, method='previous_fit', 
                                                         prev_left_line=left_line, prev_right_line=right_line, produce_out_img=False)

    undistorted_overlayed = draw_lane_on_image(img, binary_warped, g_ploty, left_line.best_fitx, right_line.best_fitx)
    
    curverad = (left_line.radius_of_curvature_m + right_line.radius_of_curvature_m) / 2
    
    lane_center_px = get_lane_center_in_pixels(g_ploty, left_line.best_fit, right_line.best_fit)
    lane_offset_m = get_lane_offset_in_meters(binary_warped.shape[1], lane_center_px)

    screen = compose_basicScreen(undistorted_overlayed, curverad=curverad, offset=lane_offset_m)

    return screen, left_line, right_line

In [ ]:
# 7-19 Process the project video and save the results.

left_line = None
right_line = None

def lane_line_basic(img):
    global left_line, right_line
    screen, left_line, right_line = visualize_lane_using_basicScreen(img, left_line, right_line)
    return screen

def writeout_lane_finding_video_with_basicScreen(src, dst, start=0, end=0):
    clip = VideoFileClip(src).subclip(start, end)
    diag_clip = clip.fl_image( lane_line_basic )
    diag_clip.write_videofile(dst)

# UNCOMMENT TO RUN
#writeout_lane_finding_video_with_basicScreen('project_video.mp4', 'project_video_basic.mp4', start=0, end=None)

# EXPERIMENTS

In [ ]:
def visualize_composite_2x3(img):
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    #sv = np.divide(np.add(hsv[:,:,1], hsv[:,:,2]), 2).astype(np.uint8)
    lsadd = np.add(hls[:,:,1], hls[:,:,2])
    ls = np.uint8(255 * lsadd/np.max(lsadd))
    
    #svrgb = cv2.cvtColor(sv, cv2.COLOR_GRAY2RGB)
    lsrgb = cv2.cvtColor(ls, cv2.COLOR_GRAY2RGB)
    #lsrgb = to_RGB(ls)
    
    h, s, v = cv2.split(hsv)
    h.fill(0)
    #s.fill(0)
    #v.fill(0)
    hsv_test = cv2.merge([h, s, v])
    hsv_test_rgb = cv2.cvtColor(hsv_test, cv2.COLOR_HSV2RGB)

    #hsv0 = cv2.cvtColor(hsv[:,:,0], cv2.COLOR_GRAY2RGB)
    hsv1 = cv2.cvtColor(hsv[:,:,1], cv2.COLOR_GRAY2RGB)
    hsv2 = cv2.cvtColor(hsv[:,:,2], cv2.COLOR_GRAY2RGB)
    
    #hls0 = cv2.cvtColor(hls[:,:,0], cv2.COLOR_GRAY2RGB)
    hls1 = cv2.cvtColor(hls[:,:,1], cv2.COLOR_GRAY2RGB)
    hls2 = cv2.cvtColor(hls[:,:,2], cv2.COLOR_GRAY2RGB)
    
    return compose_2x3_screen(diag1=hsv_test_rgb, diag2=hsv1, diag3=hsv2, 
                              diag4=lsrgb, diag5=hls1, diag6=hls2, 
                              title1="svrgb", title2="S", title3="V", 
                              title4="lsrgb", title5="L", title6="S")

In [ ]:
def demonstrate_composite_2x3_screen():

    for img in extract_frames('project_video', start_time=41.2, interval=1/24, max_images=1):

        screen = visualize_composite_2x3(img)

        plt.figure(figsize=(20,12))
        plt.imshow(screen)
        plt.show()

# UNCOMMENT TO RUN
demonstrate_composite_2x3_screen()

In [ ]:
def writeout_hsv_hls_comparison_video(src, dst, start=0, end=0):
    clip = VideoFileClip(src).subclip(start, end)
    diag_clip = clip.fl_image( composite_2x3 )
    diag_clip.write_videofile(dst)

# UNCOMMENT TO RUN
#writeout_hsv_hls_comparison_video('project_video.mp4', 'project_video_hsv_hls.mp4', start=0, end=None)